https://github.com/ftsrg/chc2c
extraction

https://arxiv.org/abs/2505.13454 pyeb: A Python Implementation of Event-B Refinement Calculus https://github.com/ncatanoc/pyeb

Compete in smtlib / cade?

TPTP parser? 

Try ingesting all of TPTP problem set. Egraph?


float16 in numpy. Brute force check properties of floats.
float8 would be a lookup table maybe.



# Jules jacobs macros vs internal

https://www.reddit.com/r/ProgrammingLanguages/comments/1aigns2/discussing_isabellehol_a_proof_assistant_for/kova7l5/

"This is a good question.

The advantage of higher-order logic over first-order logic is that propositions (and predicates) become first-class objects that you can abstract over in the theory.

Similarly, the advantage of type theory over higher-order logic is that types (and type constructors) become first-class objects that you can abstract over in the theory.

In that sense, if you understand what higher order logic gives you over first order logic, then you can understand what type theory gives you over higher order logic by analogy.

In both cases you can work around these limitations to some extent by using the meta-language as a macro system. For instance, if you only had first-order logic but you wanted to write a lemma that says something about a predicate (say, the strong induction lemma for natural numbers that is generic over some predicate P : nat -> prop), you could write the lemma as a macro that takes as input a predicate represented as a meta-level function that produces a formula, and write the proof of the lemma as a macro that generates a new proof for each new predicate P that the lemma is used on.

Similarly, in HOL you cannot write down code that is generic over a type constructor, e.g. code generic over a monad. Nor can you write down lemmas about such generic code. You can write a meta-level macro that does it, similar to how you can work around the limitations of first-order logic regarding propositions and predicates.

The big problem with this workaround is that now all your code and lemmas are checked at the use site instead of the definition site, similar to how C++ templates work. This also means that you can never write down and prove such a generic theorem; you can only generate and check instances of it.

The other main way to work around these problems is to not work in first order logic, but instead work in some theory built on top of first order logic. That's what ZFC set theory does. You can do something analogous in HOL, by not really working with types at all, but rather work in some sufficiently general un(i)typed universe and always work with predicates/subsets of that."



In [ ]:
# diff

# Hoare



In [15]:
ast.dump(ast.parse("x,y = 3,2; y = 7"))

"Module(body=[Assign(targets=[Tuple(elts=[Name(id='x', ctx=Store()), Name(id='y', ctx=Store())], ctx=Store())], value=Tuple(elts=[Constant(value=3), Constant(value=2)], ctx=Load())), Assign(targets=[Name(id='y', ctx=Store())], value=Constant(value=7))], type_ignores=[])"

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Hoare():
    pre : smt.BoolRef
    cmd : ast.AST | list[ast.AST]
    post : smt.BoolRef
    reasons : list
    def __repr__(self):
        if isinstance(self.cmd, list):
            cmd = ast.Module(body=self.cmd)
        else:
            cmd = self.cmd
        return f"{{{self.pre}}} {cmd} {{{self.post}}}"
    @classmethod
    def skip(cls, P):
        return cls(P, ast.Pass(), P, ["skip"])
    @classmethod
    def assign(cls, cmd, P):
        match cmd:
            case ast.Assign(targets=[ast.Name(id=x, ctx=ast.Store())], value=e):
                e = kdrag.reflect.expr(e)
                x = smt.Const(x, e.sort())
                return cls(smt.substitute(P, (x, e)), cmd, P, ["assign"])
            case _:
                raise ValueError(f"Unsupported assignment: {ast.unparse(cmd)}")
    def __matmul__(self, other: 'Hoare') -> 'Hoare':
        if not self.post.eq(other.pre):
            raise ValueError(f"Postcondition {self.post} does not match precondition {other.pre}")
        return Hoare(self.pre, [self.cmd, other.cmd], other.post, [self, other])





def wp(stmt : ast.AST, P : smt.BoolRef) -> Hoare:
    match stmt:
        case ast.Assign():
            return Hoare.assign(stmt, P)
        case ast.Pass():
            return Hoare.skip(P)
        case ast.Module(body=stmts):
            return wps(stmts, P)

def wps(stmts : list[ast.AST], P : smt.BoolRef) -> Hoare:
    for stmt in reversed(stmts):
        pf = wp(stmt, P)
        P = pf.pre
    return pf


def wp_sr(s : str, P : smt.BoolRef) -> Hoare:
    """
    Compute the weakest precondition for a string of Python code `s` with postcondition `P`.
    """
    tree = ast.parse(s)
    return wps(tree.body, P)


x,y,z = smt.Ints("x y z")




# python dsl


In [ ]:
import ast
from kdrag.all import *
import inspect
def datatype(s : str, locals=None, globals=None) -> smt.DatatypeSortRef:
    if locals is None:
        locals = inspect.currentframe().f_back.f_locals
    if globals is None:
        globals = inspect.currentframe().f_back.f_globals
    mod = ast.parse(s)
    body = mod.body
    if len(body) != 1:
        raise ValueError(f"Expected a single type alias, got {ast.unparse(mod)}")
    type_alias = body[0]
    if not isinstance(body[0], ast.TypeAlias):
        raise ValueError(f"Expected a single type alias, got {ast.unparse(body[0])}")
    print(ast.dump(type_alias))
    dt = kd.Inductive(type_alias.name.id)
    todo = [type_alias.value]
    while todo:
        match todo.pop():
            case ast.BinOp(op=ast.BitOr(), left=left, right=right):
                todo.append(right)
                todo.append(left)
            case ast.Name(id=name):
                dt.declare(name)
            case ast.Call(func=ast.Name(id=name), args=args, keywords=[]):
                dt.declare(name, *[( f"{name}_{n}" ,locals[arg.id]) for n,arg in enumerate(args)]) # caller_frame = inspect.currentframe().f_back
            case ast.Call(func=ast.Name(id=name), args=[], keywords=keywords):
                dt.declare(name, *[(kw.arg ,locals[kw.value.id]) for kw in keywords]) # caller_frame = inspect.currentframe().f_back
            case _:
                raise ValueError(f"Unexpected subexpresison: {ast.unparse(type_alias.value)}")
    return dt.create()
    




Int = smt.IntSort()
Real = smt.RealSort()
Foo = datatype("type Foo = Biz | Bar | Baz(Int, Int) | Boz(x = Int, y = Int)")
# Foo = datatype("type Foo = Biz | Bar | Baz(Int, Int) | Boz(x = Int, y = {Int})") explicitly mark the things that should be interpolate in?
Foo.Biz
Foo.Baz.domain(1)
Foo.x.range()




TypeAlias(name=Name(id='Foo', ctx=Store()), type_params=[], value=BinOp(left=BinOp(left=BinOp(left=Name(id='Biz', ctx=Load()), op=BitOr(), right=Name(id='Bar', ctx=Load())), op=BitOr(), right=Call(func=Name(id='Baz', ctx=Load()), args=[Name(id='Int', ctx=Load()), Name(id='Int', ctx=Load())], keywords=[])), op=BitOr(), right=Call(func=Name(id='Boz', ctx=Load()), args=[], keywords=[keyword(arg='x', value=Name(id='Int', ctx=Load())), keyword(arg='y', value=Name(id='Int', ctx=Load()))])))


Int

In [ ]:
def setcomp(s:str, locals=None, globals=None): ...

ast.parse("{{x + 1 for x in A if x > 0}}")
#smt.Lambda([y], smt.Exists([x], y == x + 1))

In [ ]:
case ast.SetComp(elt=elt, generators=generators):
    res = rec(elt)
    y = smt.FreshConst(res.sort())
    genvars = []
    smt.Lambda([y], smt.Exists())
    xs = [ast.unparse(x) for x in elt.generators[0].iter]

In [ ]:
import ast

print(ast.dump(ast.parse("{x for x in Int for y in Int if x > 0 and y < 0}"), indent=4))




Module(
    body=[
        Expr(
            value=SetComp(
                elt=Name(id='x', ctx=Load()),
                generators=[
                    comprehension(
                        target=Name(id='x', ctx=Store()),
                        iter=Name(id='Int', ctx=Load()),
                        ifs=[],
                        is_async=0),
                    comprehension(
                        target=Name(id='y', ctx=Store()),
                        iter=Name(id='Int', ctx=Load()),
                        ifs=[
                            BoolOp(
                                op=And(),
                                values=[
                                    Compare(
                                        left=Name(id='x', ctx=Load()),
                                        ops=[
                                            Gt()],
                                        comparators=[
                                            Constant(value=0)]),
               

In [2]:
from kdrag.reflect import expr
from kdrag.all import *
x = smt.Int("x")
expr("x + 1")
expr("{x for x in int if x > 0}")

ValueError: ('Could not interpret expression', '{x for x in int if x > 0}', "SetComp(elt=Name(id='x', ctx=Load()), generators=[comprehension(target=Name(id='x', ctx=Store()), iter=Name(id='int', ctx=Load()), ifs=[Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[Constant(value=0)])], is_async=0)])")

# unapply

What about a table to register unapply patterns



In [ ]:
unapply = {}

def add_unapply(t):
    
#def add_unapply(pat, t): ?

# recfunction translate

Maybe I should just dump the definitions into the main ctx. Why not?
Well, sometimes I want to hide them. I dunno.
Be an interesting experiment.

In [ ]:
smt.RecAddDefinition

In [ ]:
from kdrag.all import *
def define_rec(name, args, body):
    """
    Define a recursive function with the given name, arguments, and body.
    """
    f_not_rec = kd.define(name, args, body)
    f = smt.RecFunction(name, *[arg.sort() for arg in args], body.sort())
    smt.RecAddDefinition(f, args, smt.substitute_funs(body, (f_not_rec, f(*[smt.Var(n, f.domain(n)) for n in range(f.arity())]))))
    return f_not_rec #?
n,m = smt.Ints("n m")
add = smt.RecFunction("add", smt.IntSort(), smt.IntSort(), smt.IntSort())

add1 = define_rec("add", [n, m], smt.If(n <= 0, m, add(n - 1, m) + 1))

smt.simplify(add(3,4))

#smt.simplify(add1(3,4))


7

In [2]:
import z3
z3.z3core.Z3_to_func_decl

<function z3.z3core.Z3_to_func_decl(a0, a1, _elems=<z3.z3core.Elementaries object at 0x7d4458071cd0>)>

In [19]:
from kdrag.all import *
import kdrag.theories.nat as nat

ctx = smt.Context()
z3.ZE_DEBUG = False

subst = []
for f, defn in kd.kernel.defns.items():
    #print(f)
    f = f.translate(ctx)
    f.ast = z3.z3core.Z3_to_func_decl(f.ctx.ref(), f.ast) #f.ast.translate(ctx)
    args = [arg.translate(ctx) for arg in defn.args]
    body = defn.body.translate(ctx)
    frec = smt.RecFunction(f.name(), *[f.domain(i) for i in range(f.arity())] ,f.range())
    #print(type(f), type(body))
    subst.append((f, frec(*[smt.Var(f.arity() - n - 1, f.domain(n)) for n in range(f.arity())])))
    subst[0][0].as_func_decl()
    smt.substitute_funs(body, *subst)
print(subst)
for f, defn in kd.kernel.defns.items():
    #print(f)
    f = f.translate(ctx)
    print(defn.args)
    for arg in defn.args:
        print(arg)
        print(arg.translate(ctx))
    args = [arg.translate(ctx) for arg in defn.args]
    print(args)
    body = defn.body.translate(ctx)
    frec = smt.RecFunction(f.name(), *[f.domain(i) for i in range(f.arity())] ,f.range())
    print(frec, args, smt.substitute_funs(body, *subst))
    smt.RecAddDefinition(frec, args, smt.substitute_funs(body, *subst))


def simp(t):
    t = t.translate(ctx)
    print(t)
    return smt.simplify(smt.substitute_funs(t, *subst))



simp(nat.double(nat.Z, nat.Z))







[(even, even(Var(0))), (odd, odd(Var(0))), (safe_pred, safe_pred(Var(0))), (from_int, from_int(Var(0))), (to_int, Var(0)), (double, double(Var(0))), (add, add(Var(1), Var(0)))]
[Int]
Int
even
[even]
even [even] Exists(y, odd(even, 2*y))
[Int]
Int
even
[even]
odd [even] Exists(y, odd(even, 2*y + 1))
[n]
n
Select
[Select]
safe_pred [Select] If(is(Z, Select), x!1, pred(Select))
[a]
a
a
[a]
from_int [a] If(a <= 0, x!1, S(from_int(a - 1)))
[n]
n
n
[n]
to_int [n] If(is(Z, n), 0, 1 + (pred(n)))
[n]
n
n
[n]
double [n] If(is(Z, n), x!1, S(S(double(pred(n)))))
[x, y]
x
x
y
x
[x, x]
add [x, x] If(is(Z, x), x, S(add(x, pred(x))))


Z3Exception: b'index out of bounds'

# Extract Langs
C boehm GC
Rust via maturin + RC
Separate z3 context + recfunction
Java py4j https://www.py4j.org/



Maybe the rust version could lay some claim to
Both fast and correct version of code.
Output Kani conditions.


https://docs.rs/numpy/latest/numpy/

copilot inspired.
State loop?







## rust

What about kdrag-rs? Just do it all in rust. Learning rust details could be fun


Could macro the shit out of it I suppose?



In [ ]:
%%file /tmp/test.rs
---cargo
[dependencies]
z3 = "0.12.1"
lazy_static = "1.5.0"
---

use z3::ast::Ast;
/*

use std::sync::LazyLock;

lazy_static::lazy_static! {
    static ref Z3_CTX: z3::Context = z3::Context::new(&z3::Config::new());
}

static Z3_CTX: LazyLock<z3::Context> = LazyLock::new(|| {
    z3::Context::new(&z3::Config::new())
 });


fn pmatch<'ctx>(ctx : &'ctx z3::Context, vs : , fm : &'ctx z3::ast::Dynamic<'ctx>, pat : &'ctx z3::ast::Dynamic<'ctx>) -> Vec<> {
    let mut todo = vec![(fm, pat)];
    while todo{

    }
}
*/

#[derive(Debug)]
struct Proof<'ctx> {
    fm : &'ctx z3::ast::Bool<'ctx>,
}
fn prove<'ctx>(ctx : &'ctx z3::Context, fm : &'ctx z3::ast::Bool) -> Proof<'ctx> {
    let solver = z3::Solver::new(ctx);
    solver.assert(&fm.not());
    if solver.check() == z3::SatResult::Unsat {
        Proof { fm }
    } else {
        panic!("Unsatisfiable");
    }
}


fn main(){
    let ctx = z3::Context::new(&z3::Config::new());
    let solver = z3::Solver::new(&ctx);
    let x = z3::ast::Int::new_const(&ctx, "x");
    let y = z3::ast::Int::new_const(&ctx, "y");
    let z = z3::ast::Int::new_const(&ctx, "z");

    let _true = z3::ast::Bool::from_bool(&ctx, true);
    let p = prove(&ctx, &_true);
    println!("Proof: {:?}", p.fm);

    let _T = z3::Sort::uninterpreted(&ctx, z3::Symbol::from("T"));


    solver.assert(&x._eq(&y));
    solver.assert(&x._eq(&(y + z)));

    if solver.check() == z3::SatResult::Sat {
        println!("Satisfiable");
        println!("Model: {:?}", solver.get_model());
    } else {
        println!("Unsatisfiable");
    }

}

Overwriting /tmp/test.rs


In [119]:
! cargo +nightly -Zscript /tmp/test.rs

   Compiling test- v0.0.0 (/tmp/test.rs)
  --> test.rs:47:9
   |
47 |     let _T = z3::Sort::uninterpreted(&ctx, z3::Symbol::from("T"));
   |         ^^ help: convert the identifier to snake case: `_t`
   |
   = note: `#[warn(non_snake_case)]` on by default

    Finished `dev` profile [unoptimized + debuginfo] target(s) in 0.10s
     Running `/home/philip/.cargo/target/e8/a8b1c02d8deab1/debug/test-`
Proof: true
Satisfiable
Model: Some(z -> 0
x -> 0
y -> 0
)


In [30]:
import maturin

ModuleNotFoundError: No module named 'maturin'

In [ ]:
class RustModule():
    def __init__(self, modname):
        self.modname = modname
        self.funcs = {}
        self.types = []

    def add_type(self, type_code):
        self.types.append(type_code)
    
    def add_function(self, fun_name, fun_code):
        """
        Add a function to the module with the given name and code.
        """
        if fun_name in self.funcs:
            raise ValueError(f"Function {fun_name} already exists in module {self.modname}.")
        self.funcs[fun_name] = fun_code


    def __str__(self):
            types = "\n".join("#[derive(FromPyObject, IntoPyObject)]\n" + type_code for type_code in self.types)
            fun_code = "\n".join(f"#[pyfunction]\n{fun_code}" for fun_code in self.funcs.values())
            add_funs = "\n".join(f"m.add_function(wrap_pyfunction!({fun_name}, m)?)?;" for fun_name in self.funcs.keys())
            return f"""\
use pyo3::prelude::*;
use pyo3::wrap_pyfunction;
use pyo3_ffi::c_str;

{types}

{fun_code}

#[pymodule]
fn {self.modname}(m: &Bound<'_, PyModule>) -> PyResult<()> {{
    {add_funs}
    Ok(())
}}
"""

M = RustModule("kdrag_smt")
M.add_function("add", "fn add(n: i32, m: i32) -> i32 {n + m}\n}")
M.add_type("struct MyType { field1: i32, field2: String }")
print(str(M))

use pyo3::prelude::*;
use pyo3::wrap_pyfunction;

#[derive(FromPyObject, IntoPyObject)]
struct MyType { field1: i32, field2: String }

#[pyfunction]
fn add(n: i32, m: i32) -> i32 {n + m}
}

#[pymodule]
fn kdrag_smt(m: &Bound<'_, PyModule>) -> PyResult<()> {
    m.add_function(wrap_pyfunction!(add, m)?)?;
    Ok(())
}



## wasm

Extract to wasm, run it./

Hmm. wasmtime has gc... That sweetens the pie a lot.

https://developer.mozilla.org/en-US/docs/WebAssembly
https://developer.mozilla.org/en-US/docs/WebAssembly/Guides/Understanding_the_text_format

https://github.com/WebAssembly/gc/blob/main/proposals/gc/Overview.md
https://github.com/bytecodealliance/wasmtime-py

https://tanishiking.github.io/posts/wasm-gc/

Codegen an ematcher. We used to talk about this. Or codegenning actions.

Hmm. But maybe one can hold external references to the host... so maybe structured objects could be dealt with that way. Using python.

Or pretty print, process call wasmtime cli. Read output. Could be worse. Gets me ready for the day wasm_gc is avaiable in the bindings.
wasmer?



In [44]:
from wasmtime import Store, Module, Instance, Func, FuncType

store = Store()
module = Module(store.engine, """
  (module
    (func $hello (import "" "hello"))
    (func (export "run") (call $hello))
  )
""")

def say_hello():
    print("Hello from Python!")
hello = Func(store, FuncType([], []), say_hello)

instance = Instance(store, module, [hello])
run = instance.exports(store)["run"]
run(store)

Hello from Python!


In [ ]:
from wasmtime import Store, Module, Instance

store = Store()
module = Module(store.engine, """
  (module
    (func $myadd (export "myadd") (param i32 i32) (result i32)
    (local.get 0)
    (local.get 1)
    (i32.add)
  )
  )
""")
instance = Instance(store, module, [])
myadd = instance.exports(store)["myadd"]
myadd(store,1,2)

3

In [4]:
%%file /tmp/test.wat

  (module
    (type $point (struct (field $x i64) (field $y i64)))
    (func $myadd (export "myadd") (param i32 i32) (result i32)
    (local.get 0)
    (local.get 1)
    (i32.add)
  )
  )
  

Writing /tmp/test.wat


In [ ]:
! wasmtime compile --wasm gc /tmp/test.wat -o /tmp/test.wasm

/bin/bash: line 1: wasmtime: command not found


In [6]:
from wasmtime import Store, Module, Instance, Config, Engine
config = Config()
config.wasm_gc = True
engine = Engine(config)
store = Store(engine)
module = Module.deserialize_file(store.engine, "/tmp/test.wasm")

WasmtimeError: failed deserialization for: /tmp/test.wasm

Caused by:
    Module was compiled with support for WebAssembly feature `gc` but it is not enabled for the host

In [3]:
from wasmtime import Store, Module, Instance, Config, Engine
config = Config()
config.wasm_gc = True
engine = Engine(config)
store = Store(engine)
module = Module(store.engine, """
  (module
    (type $point (struct (field $x i64) (field $y i64)))
    (func $myadd (export "myadd") (param i32 i32) (result i32)
    (local.get 0)
    (local.get 1)
    (i32.add)
  )
  )
""")
instance = Instance(store, module, [])
myadd = instance.exports(store)["myadd"]
myadd(store,1,2)

WasmtimeError: failed to parse WebAssembly module

Caused by:
    struct indexed types not supported without the gc feature (at offset 0xb)

In [ ]:
def struct_of_dt(d : DatatypeSortRef):
    for c in d:
        field = " ".join(f"(field ${f.name()} {f.sort()}" for f in c.fields())
        cons = f"(struct ${d}.{c.name()} "
    return f"""
    (rec
        
    )
    """




In [ ]:
def to_wasm_type(sort : smt.SortRef):
    if sort == smt.BitVec(32):
        return "i32"
    elif sort == smt.BitVec(64):
        return "i64"

def wasm_of_expr(e : smt.ExprRef):
    # recurse through
    if 

    decl = e.decl()
    if e.is_add()
    



def to_wasm(name, args, body : smt.ExprRef):
    """
    Convert a SMT expression to a WebAssembly function.
    """
    return f"""
(module
  (func ${name} (export {name}) {params} (result {to_wasm_type(body.sort())})
    
  )

    """

# combinators

A full set of combinators

Proof terms that I translate?

lambda([P], P)


APL is master of the ocmbiunators. An APL knuckledragger would use a heavy combinator style.
Knuckledragger numpy?
vectorized proofs

https://en.wikipedia.org/wiki/Combinatory_logic
https://combinatorylogic.com/index.html


forall xs, smt.And() => smt.Or()   normal form
forall xs, smt.And() => conc   normal form

@ * + ~ << >> combinators for Proofs? Hook and etc.
Kind of a fun idea. ssreflect?



Harrison book has some

pluck And(xs) => P  ====>   x[n] => And(xs - x[n]) => P
insert   x => And(xs) => P  ====> And(xs + x + xs) => P
sort And(xs) => P ====> And(sorted(xs)) => P






In [ ]:


def cut(a, b):

In [ ]:
def curry(pf : kd.Proof) -> kd.Proof:
    assert isinstance(pf, kd.Proof)
    assert smt.is_implies(pf.thm)
    hyp, conc = pf.thm.children()
    assert smt.is_and(hyp)
    args = hyp.children()
    return kd.axiom(smt.Implies(smt.And(args[:-1]), smt.Implies(args[-1], conc)), by=["curry", pf])
a,b,c = smt.Bools("a b c")
curry(kd.prove(smt.Implies(smt.And(a, b, c), a)))

# curryn ?

def uncurry(pf : kd.Proof) -> kd.Proof:
    assert isinstance(pf, kd.Proof)
    assert smt.is_implies(pf.thm)
    hyp, conc = pf.thm.children()
    assert smt.is_implies(conc)
    hyp1, conc = conc.children() 
    return kd.axiom(smt.Implies(smt.And(hyp, hyp1), conc), by=["uncurry", pf])
uncurry(curry(kd.prove(smt.Implies(smt.And(a, b, c), a))))

# modus is apply





|- Implies(And(And(a, b), c), a)

Design  as decorator?

A little weird. Interesting looking patterns.

```
def Ex(vs, P):
   def res(cb):

def ExistsE(vs, P):
    


def ForAllI(vs, P):
    vs, ab = kd.kernel.herb(smt.ForAll([vs], P))

def Pi(vs, P):
    def res(cb):
        vs, ab = kd.kernel.herb(smt.ForAll(vs, P))
        return modus(ab, cb(*vs))
    return res

@Pi([n], foo(n))
def step1(goal, vs)


@fixes(smt.ForAll([n], foo(n)))
def foo(goal, n):
    @fixes
    def res(goal1, m):
        return kd.prove(...)
    return res
```

In [25]:
from typing import Callable
from functools import partial
#@partial
def fixes(goal : smt.QuantifierRef, cb : Callable[[smt.ExprRef,...], kd.Proof]) -> kd.Proof:
    vs, ab = kd.kernel.herb(goal)
    a = cb(*vs) # ab.thm.arg(0) pass in?
    pf = kd.kernel.modus(ab, a)
    assert pf.thm.eq(goal)
    return pf

foo = smt.Function("foo", smt.IntSort(), smt.BoolSort())
n = smt.Int("n")
pf1 = kd.axiom(smt.ForAll([n], foo(n)))
fixes(smt.ForAll([n], foo(n)), lambda n: pf1(n))



|- ForAll(n, foo(n))

In [ ]:
def aprove(goal, instans):
    intros(goal, lambda goal1, *args: kd.prove(goal1, by=instans(*args)))

# BHK 
simp is definitional equality.
Model
ForAllI(lambda x: f)
Pi()
ForAllI([x,y,z], lambda x,y,z: kd.prove(    ))

def ForAllI(vs, prop):
    def res(f):
        vs, lemma = herb(ForAll(vs, prop))
        p = f(lemma.arg(0), *vs)
        return modus(lemma, p)
    return res
    #def res(goal, *vs):

class CtxProof():
    ctx : 
    thm : smt.
    pf : kd.Proof
class CtxExpr():


def prove(fm):
    if hasattr(fm, "kdrag_ctx"):
        return prove(smt.ForAll(smt.Implies(fm.kdrag_ctx, fm)))
    else:
        return prove(fm)




def ImplI(prop):
    def res(f):
        p = prop.arg(1)
        p.kdrag_ctx = prop.arg(0) + prop.kdrag_ctx
        f(p)

        f(CtxProof())
        ? = ?(prop)


@ForAllI([x,y,z])
def add_comm(x : int, y : int, z : int):
    return kd.prove()

def forall(f):
    inspect()
    kd.kernal.herb

@forall
def add_comm(x : int, y : int, z : int):

type Prop = BoolRef
def ind_eq(motive : Callable([ExprRef,ExprRef], BoolRef), a, b, p : kd.Proof):
    t1 = motive(a,a)
    assert p.thm.eq(t1)
    t2 = motive(a,b)

def ImplI()


contextmanager
generators for algerbaic effects

def prove(p):
    ctx = get_ctx()
    kd.prove(ctx => )

global? ctx

l = Lemma(forall )
with l.fix() as y:
    with z.intros() as ih1:



# protocol

Protocls as replacement for some uses of typeclasses

As replacement for ExprRef

metatheory.jl had some interface.

Lazy Expr
def to_smt(self) -> smt.ExprRef: ...



In [ ]:
class Ast(Protocol):
    def arg(self, i: int) -> 'Ast': ...
    def children(self) -> list['Ast']: ...
    def eq(self, other: 'Ast') -> bool: ...
class Sort(Ast, Protocol):
    def name(self) -> str: ...
class FuncDecl(Ast, Protocol):
    def name(self) -> str: ...
    def arity(self) -> int: ...
    def domain(self, i: int) -> 'Sort': ...
    def range(self) -> 'Sort': ...
class Expr(Ast, Protocol):
    def __eq__(self, other: 'Expr') -> "Expr": ...
    # all operations?
    # is_add: ... operations. Hmm.




In [ ]:
from typing import Protocol, runtime_checkable
from kdrag.all import *

@runtime_checkable
class Monoid(Protocol):
    add: smt.FuncDeclRef
    zero: smt.ExprRef
    assoc: kd.Proof # |- forall x, y, z: add(add(x, y), z) == add(x, add(y, z))
    lunit: kd.Proof # |- forall x: add(zero, x) == x
    runit: kd.Proof # |- forall x: add(x, zero) == x

@runtime_checkable
class Group(Monoid, Protocol):
    inv: smt.FuncDeclRef # inverse operation
    inv_lunit: kd.Proof # |- forall x: add(inv(x), x) == zero
    inv_runit: kd.Proof # |- forall x: add(x, inv(x)) == zero

# alternative to SortDispatch
ExprRef.__add__ = lambda x,y: x.sort().add(x,y)


@runtime_checkable
class Assoc(Protocol):
    assoc: kd.Proof


add = (x + y).decl()
add.assoc = kd.prove(smt.ForAll([x, y, z], (x + y) + z == x + (y + z)))
isinstance(add, Assoc)


In [ ]:
Lattice = smt.DeclareSort("Lattice")
meet = smt.Function("meet", Lattice, Lattice, Lattice)
kd.notation.and_.register(Lattice, meet)
join = smt.Function("join", Lattice, Lattice, Lattice)
kd.notation.or_.register(Lattice, join)
x,y,z = smt.Consts("x y z", Lattice)
meet_assoc = kd.axiom(smt.ForAll([x,y,z], x & (y & z) == (x & y) & z))
meet_comm = kd.axiom(smt.ForAll([x,y], x & y == y & x))
meet_idem = kd.axiom(smt.ForAll(x, x & x == x))
join_assoc = kd.axiom(smt.ForAll([x,y,z], x | (y | z) == (x | y) | z))
join_comm = kd.axiom(smt.ForAll([x,y], x | y == y | x))
join_idem = kd.axiom(smt.ForAll(x, x | x == x))

#kd.notation.le([x,y], x & y == x)


# complete
sup = smt.Function("sup", set_.SetSort(Lattice), Lattice)
inf = smt.Function("inf", set_.SetSort(Lattice), Lattice)


monotone = smt.Function("monotone", smt.ArraySort(Lattice, Lattice), smt.BoolSort())


# https://en.wikipedia.org/wiki/Knaster%E2%80%93Tarski_theorem
gfp = sup(smt.Lambda([x], x <= f(x)))
lfp = inf(smt.Lambda([x], f(x) <= x))






# Fixed point

Could spacer or BMC be a good 

Could I use this for a typeclass like system?

for subtypes?

from kdrag.all import *

fp = smt.FixedPoint()
fp.set(engine="bmc")



# user propagators
?
I have some ideas about forall x, inj(proj(x)) = x
User propagators good for that maybe? A simple application.
is_set(smt.ArraySort())





# subtyping





In [35]:
from kdrag.all import *
type Telescope = list[tuple[smt.ExprRef, smt.BoolRef] | tuple[smt.ExprRef, smt.ArrayRef] | smt.ExprRef]
def TForAll(xs : Telescope, P : smt.BoolRef) -> smt.BoolRef:
    """
    Dependent forall quantifier for a telescope of variables.
    """
    for v in reversed(xs):
        if isinstance(v, tuple):
            (v, T) = v
            if T.sort() == smt.BoolSort():
                P = kd.QForAll([v], T, P)
            elif isinstance(T, smt.ArrayRef) or (isinstance(T, smt.QuantifierRef) and T.is_lambda()):
                P = kd.QForAll([v], T(v), P)
            else:
                raise TypeError(f"Unsupported type for quantifier: {T}")
        else:
            P = kd.QForAll([v], P)
    return P
x, y, z = smt.Reals("x y z")
kd.prove(TForAll([(x, x > 0), (y, y > x)], y > -1))
Pos = smt.Lambda([x], x > 0)
def GT(x):
    return smt.Lambda([y], y > x)
kd.prove(TForAll([(x, Pos), (y, GT(x))], y > -1))





|- ForAll(x, Implies(x > 0, ForAll(y, Implies(y > x, y > -1))))

https://fstar-lang.org/tutorial/proof-oriented-programming-in-fstar.pdf

In [ ]:
Prop = kd.Enum("Prop", ["tt"])
p = smt.Const("p", Prop)

[(p, x == y)] #==> forall p: x == y
def Eq(x,y):
    """
    Equality as a dependent type.
    """
    return smt.Lambda([p], x == y)
def Squash(b):
    p = smt.FreshConst(Prop)
    return smt.Lambda([p], b)


In [ ]:
n = smt.IntSort()
Nat = smt.Lambda([n], n >= 0)
Zero = smt.Lambda([n], n == 0)
Pos = smt.Lambda([n], n > 0)
Neg = smt.Lambda([n], n < 0)
Even = smt.Lambda([n], n % 2 == 0)
Odd = smt.Lambda([n], n % 2 == 1)

incr = kd.define("incr", [n], n + 1)
kd.pdefine("incr", [(n, Nat)], Nat, n + 1)
kd.pdefine("incr", [(n, Nat)], smt.Lambda([m], m > n), n + 1)


https://www.philipzucker.com/notes/Languages/ocaml/#universal-types

https://okmij.org/ftp/ML/trep.ml

Boxing and unboxing.
My notes on "Any"  in knuckledragger

Hmm. Maybe I could do a single RecFunction for my universe levels. Then I do get to bake it in to z3.
Or for open recursion?

I do have existentials (?) Oh myyyyyy

Box = kd.Record("Box", (typ : 'a Code, val : TypeVar("a")))
Hmm. Yeah, I kind of need GADTs to use the tag trick.



def Get


https://dl.acm.org/doi/10.1145/3649836 Persimmon: Nested Family Polymorphism with Extensible Variant Types. Poster at NEPLS



GADTs.

PInductive()
  pred=
  pred=

InductiveRel
wf
Why did I call it rel and not use the wf system?
That's weird.



In [ ]:
import kdrag.theories.seq as seq

def Vec(n, S):
    T = seq.Seq(S)
    x = smt.FreshConst(T)
    return smt.Lambda([x], smt.Length(x) == n)

def IVec(n):
    return Vec(n, smt.IntSort())
n = smt.Int("n")
Nat = smt.Lambda([n], n >= 0)
x,y = smt.Consts("x y", seq.Seq(smt.IntSort()))
tdefine("add", [(n, Nat), (x, IVec(n)), (y, IVec(n))],
smt.cond(
    (n == 0, Nil),
    default=Cons(x.head + y.head, add(n - 1, x.tail, y.tail) , 
    ), IVec(n)
)

#( add(n - 1, x.tail, y.tail) ,  Nat ))

TForAll([(n, Nat), (x, IVec(n)), (y, IVec(n))],   ? )



Lambda(c!227, Length(c!227) == 2)

In [ ]:
# extensible sort of Codes
# Universes are an open type. Their codes are open
# This is exactly what the extensible tricks in ocaml are for. Keys. 'a tag

TypeCode = smt.DeclareSort("TypeCode")
# Code = smt.IntSort()
# El = OpenFunction()


def El(code):



In [ ]:
# Naaaaaah, doesn't really make sense.
def ann(x, T):
    undef = FreshConst(x.sort())
    smt.If(T(x), x, undef)

def Epsilon(T):
    FreshConst()
    smt.If(smt.Exists([y], T(y)) T(x), x, undef)


In [ ]:
type Type = smt.ArraySortRef | smt.QuantifierRef | smt.BoolRef

# domain cannot be inferred
def domain(A : TYPE):

In [ ]:
def Pi(A,B): ...
def Pi(xA, B):
    (x,A) = xA
    #B(x, f(x)))) vs B(x)(f(x)) vs B(f(x))
    # Following prior conventions, B should just be a function expression that already contains x
    # B has to be a predicate. it can't contain it's variable.
    f = smt.FreshConst(smt.ArraySort(x.sort(), B(x).domain()))
    return smt.Lambda([f], TForAll([(x,A)], B(f(x))))



Could record known substype and register different coercions


SHould All of these optionally take a Tuple?

def TForAll(vs, P):
    if isiinstance(P, tuple):
        P = P[1](P(0))

def tdefine(name, args, body):
    if isinstance(body, tuple):
    


In [ ]:
def subtype_dom(A):
    if isinstance(A, smt.QuantifierRef):
        return A.sort()

def is_subtype(A,B, by=[]):
    #x =
    return smt.Prove(A <= B, by=by)  #smt.IsSubSet()
    #return kd.prove(kd.TForAll([(x,A)], B(x)), by=by)

types are a syntactic discipline to enforce levels of abstraction

Type systems as tactic.

Homotopy type and transport and stuff probably says how to manually translate isomorphic based proofs. Which is what Talia was getting at?


Definitional equality checks use simp as subtactic.




In [ ]:




def TExists(xs : Tele, P : smt.BoolRef) -> smt.BoolRef:
    for v in reversed(xs):
        if isinstance(v, tuple):
            (v, T) = v
            if T.sort() == smt.BoolSort():
                P = kd.QExists([v], T, P)
            elif isinstance(T, smt.QuantifierRef) and T.is_lambda():
                P = kd.QExists([v], T(v), P)
            elif isinstance(T, smt.ArraySortRef):
                P = kd.QExists([v], T(v), P)
            else:
                raise TypeError(f"Unsupported type for quantifier: {T}")
        else:
            P = kd.QExists([v], P)
    return P

# TLambda
# TExists



def pdefine(name, args, out, body, by=[]):
    f = kd.define(name, args, smt.If(cond, body, undef))
    kd.prove(smt.TForAll(args, out(f(*args))))
    substypes[f] = SubTypeDefn(f, args, out, )

def subtype_tac(tele, P):
    # sweep though expression
    # prove TForAll(tele, P) via subtype abstractions.
    # Don't fail, output needed subproofs instead. Or raise with them?

subtypes = {}

# Gamma, x : T, Delta |- x : T
def refl_tele(tele, n : int) -> kd.Proof:
    (x,T) = tele[n]
    return kd.prove(smt.TForAll(tele, T(x)))




def synth(tele, t) -> tuple[Subsort, kd.Proof]:
    """
    Synthesize the type of an expression in a telescope.
    Returns a tuple of the type and a proof that the expression has that type.
    """
    if n := tele_vars(tele).find(t):
        return tele[n][1], refl_tele(tele, n)
    elif smt.is_app(t):
        decl, children = t.decl(), t.children()
        if t in subtypes:
            subtype_defn = subtypes[t]
            subproofs = [check(c, v) for c, v in zip(children, subtype_defn.tele)]
            kd.prove(smt.TForAll(subtype_defn.tele, subtype_defn.out(t(*t.children()))), by=subproofs + [subtype_defn.subtype_lemma])
            return subtype_defn.out, 
def check(tele, t, T) -> kd.Proof:


def ann(t,T):
    # refine
    # annotation is semantically an identity function. Or a refining function.
    x = FreshConst(t.sort())
    return kd.tdefine("ann", [(x,T)], T, x)

def ann(t, T):
    return has_type(?, t, T)

def restrict(f, T):
    # Can sensible annotate a term. If 
    xs = [smt.FreshConst(args) for arg in f.args]
    undef = smt.FreshFunction(*[x.sort() for x in xs], f.sort())
    return smt.Lambda([x], smt.If(T(f(*xs)), f(*xs), undef(*xs)))  # restrict f to the type T

def restrict(f, T, by=[]):
    if f in subtypes:
    # make new anonymous definition?
        return kd.tdefine(freshname, args, smt.And(T, Tf), f(), by=[f.subtype_lemma] + by)
    else:
        return kd.tdefine(freshname, args, T, f(args), by=by)

# i could tag metadata to kd.Proof objects. tele, vs. Or it could go in the reasons field... yikes.




def has_type(tele : Telescope, t : smt.ExprRef, T : smt.BoolRef, by=[], **kwargs):
    if n := tele_vars(tele).find(t):
        return refl_tele(tele, n)
    elif smt.is_app(t):
        decl, children = t.decl(), t.children()
        if decl in subtypes:
            subtype_defn = subtypes[decl]
            has_typ()


# Another case wherwe maybe my core Proof thing holding a context would be useful.
def has_type(tele : Telescope, t : smt.ExprRef, T : SubSort, by=[], **kwargs):
    # Tactic to prove TForAll(tele, T[t])
    # search through collecting up substypes.

    lemmas = [] # lemmas starts with tele somehow?
    todo = [t]
    # The let z3 handle it version.
    # The telescope character makes it annoying

    # get topological order of children and go bottom up?
    # compose ctx -> P ,  tele -> P1 moves
    # cut move   ctx -> P   ctx -> (P -> P1)  --->  ctx -> P1 
    # I need hilbert style combinators

    # Use BMC or horn solvers?
    # use consider axiom?
    while todo:
        t = todo.pop()
        if n := tele_vars(tele).find(t):
            lemmas.append(refl_tele(tele, n))

            #kd.prove(TForAll(tele, T(t)))

        if smt.is_app(t):
            decl, children = t.decl(), t.children()
            if decl in subtypes:
                subtype_defn = subtypes[decl]
                l = subtype_defn.subtype_lemma
                if len(children) > 0 and isinstance(l, smt.QuantifierRef) and l.is_forall():
                    l = l(children[0]) # One layer at least is easy
                lemmas.append(l)
                todo.extend(children)
        """
        if smt.is_app(t):
            decl,children = t.decl(), t.children()
            if decl in subtypes:
                subtype_defn = subtypes[decl]
                l = subtype_defn.subtype_lemma
                while smt.is_implies(l) or smt.is_forall(l):
                    if isinstance(l, smt.QuantifierRef) and l.is_forall():
                        c = children.pop()
                        l = l(c) # instantiate quantifier.
                    elif smt.is_implies(l):
                        goal = l.arg(0)
                        lemmas.append(kd.prove(goal, by=lemmas))
                        l = l.arg(1)
                for c in children:
                    kd.prove( )
                    l = l[c]
        """
    return kd.prove(smt.TForAll(tele, T(t)), by=by + lemmas, **kwargs) # unfold=1






SyntaxError: incomplete input (647665221.py, line 43)

idea: just using qforall would make the quantifer easier to instnatiate. Otherwise we need a 
A => forall x, B    ---> A => B[t/x] rule

iterate uncurry and 
A => forall x (B[x] => C)  ---> A => (B[t] => C) ---> and(A,B) => C

comp(acc, instan2(pf.arg(1), t)



Not all things of shape forall xs, A => B
can be easily turned into forall x0 A0 => forall x1 A1 => ,, (?)
Well. In some null sense forall x0 True, forall x1 True, ... forall xn A


Oh but I have that with compose instan2
    .  forall x phi(x) => phi(t)




QForAll([x], Nat(x), P) is fine.
But still would want to register.

Quantifiers are evil and implications are just whatever for smt. So it's not symmettric in that sense


What if cbs just get fired at the end and they delete non useful stuff.

What if GoalCtx was kept in sequent `forall xs, And(hyps) => P` normal form 

Interesting... Use lemmas as a second stack. Does this work? It is lensy right? This is the trail.

Is there a way to do an earlier call to cbs?

maybe lemmas and cbstack

## Lemma callbacks
lemma : id -> [callback]
def add_lemma(p):
    pid = p.thm.get_id()
    cbs = lemma.get(p.thm.get_id())
    if isinstance(cbs, list):
        lemmas[pid] = p
        for cb in cbs:
            cb()
    elif isinstance(cbs, kd.Proof):
        pass
    else:
        raise UnExpcted




lookup proofs by get_id
Could also close branches by lookup up to see if already proven.
That's kind of interesting.



What if I put the callbacks on the goals stack. So then I call them as soon as the subgoals are done
Goals might be an iniaprrporatye name at this point.

goals.append(newgoal)
goals.append(cb)

also might help with eexists
subst[v] = whatever
And then cb can look for it

def eexists():
    vs, body = open_binder(goal)
    for v in vs:
        self.subst[v] = None
    def cb():
        t = self.subst[v]
        kd.kernel.abstract()

    goals.append(body)
    goals.append(cb)



class Lemma2
    def intros():
    def intro()







```python
Lemmas()
    self.lemmas = {}
    cbs: list[Callable[[], None]]

    def fixes(self):
        vs, ab = kd.kernel.herb(self.goals)
        def cb():
            a = lemmas[ab.thm.arg(0).get_id()]
            #a = lemmas.pop()
            #if p.thm.arg(0)
            pb = modus(ab, a)
            lemmas[pb.thm.get_id()] = pb
            lemmas.append(modus(a, ab))
        cbs.append(cb)
        cbs.append(lambda: self.lemmas.find(pf, t.arg(0)))
    def 

    def qed(self):
        for cb in reversed(cb):
            cb()
        return lemma.find(pf, self.topgoal)
```


In [ ]:
from dataclasses import dataclass
from typing import Optional
from kdrag.all import *
from kdrag.notation import Telescope

type QCtx = tuple[list[smt.ExprRef], list[smt.BoolRef]]
type Telescope1 = list[tuple[smt.ExprRef, smt.QuantifierRef | smt.ArrayRef]]
type Telescope2 = 


@dataclass
class NormTele():
    vs : list[smt.ExprRef]
    pred : list[smt.ArrayRef | smt.QuantifierRef] # normalized into predicate form or normalize into Bool form?
    @classmethod
    def from_tele(cls, tele : Telescope) -> 'NormTele':
        vs = []
        pred = []
        for v in tele:
            if isinstance(v, tuple):
                (v, T) = v
                vs.append(v)
                if T.sort() == smt.BoolSort():
                    pred.append(smt.Lambda([v], T))
                elif isinstance(T, smt.ArrayRef) or (isinstance(T, smt.QuantifierRef) and T.is_lambda()):
                    pred.append(T)
                else:
                    raise TypeError(f"Unsupported type for quantifier: {T}")
            else:
                vs.append(v)
                pred.append(smt.Lambda([v], smt.BoolVal(True)))
        return cls(vs, pred)
    def precond(self):
        return smt.And(T(v) for v,T in zip(self.vs, self.preds))
    def eval(self, P):
        return smt.QForAll(self.vs, self.precond(), P)

#def tele_apply(pf : kd.Proof, *args) -> kd.Proof:
n,m,k = smt.Ints("n m k")
Nat = smt.Lambda([n], n >= 0)
NormTele.from_tele([(n, Nat), (m, m > 0), k])

NormTele(vs=[n, m, k], pred=[Lambda(n, n >= 0), Lambda(m, m > 0), Lambda(k, True)])

## Telescoped axioms

telescope normal form for Proofs forall x, p -> forall y, q -> ...


Qnormal form forall xs, ctx => A




In [ ]:
from kdrag.all import *
from kdrag.notation import Telescope, SubSort, TForAll, TExists
from typing import Optional
from dataclasses import dataclass

@dataclass
class SubTypeDefn():
    decl : smt.FuncDeclRef
    ctx : list[tuple[smt.ExprRef, smt.QuantifierRef | smt.ArrayRef]]
    T : Optional[smt.BoolRef]
    ax : kd.Proof # forall xs, tele(xs), out(f(xs))

def tele_vars(tele : Telescope) -> list[smt.ExprRef]:
    """
    Extract the variable names from a telescope.
    """
    return [v if isinstance(v, smt.ExprRef) else v[0] for v in tele]




def norm_tele(tele : Telescope) -> Telescope:
    acc = []
    for v in tele:
        if isinstance(v, tuple):
            (var, T) = v
            if isinstance(T, smt.ArrayRef) or (isinstance(T, smt.QuantifierRef) and T.is_lambda()):
                acc.append((var, T))
            elif T.sort() == smt.BoolSort():
                acc.append((var, smt.Lambda([var], T)))
            else:
                raise TypeError(f"Unsupported type for quantifier: {T}")
        else:
            acc.append((v, smt.Lambda([v], True)))
    return acc



def tele_cond(tele : Telescope) -> Optional[smt.BoolRef]:
    """
    Extract the condition from a telescope.
    """
    acc = []
    for v in tele:
        if isinstance(v, tuple):
            (v, T) = v
            if T.sort() == smt.BoolSort():
                acc.append(T)
            elif isinstance(T, smt.ArrayRef) or (isinstance(T, smt.QuantifierRef) and T.is_lambda()):
                acc.append(T(v))
            else:
                raise TypeError(f"Unsupported type for quantifier: {T}")
    if len(acc) == 0:
        return None
    elif len(acc) == 1:
        return acc[0]
    else:
        return smt.And(acc)

subtypes = {}

# allow synthesis of output Sort?
def tdefine(name, args : Telescope, T : SubSort, body, by=[]):
    """
    Define a subtype with a name.
    """
    tvars = tele_vars(args)
    body_lemma = kd.prove(TForAll(args, T(body)), by=by)
    undef = smt.FreshFunction(*[v.sort() for v in tvars], body.sort())
    cond = tele_cond(args)
    f = kd.define(name, tvars, smt.If(cond, body, undef(*tvars)))
    f.tele = norm_tele(args)
    f.subtype = T
    f.subtype_lemma = kd.prove(TForAll(args, T(f(*tvars))), by=[body_lemma, f.defn]) # I could do this more directly. It follows from a single unfold.
    subtypes[f] = SubTypeDefn(f, args, T, f.subtype_lemma)
    return f

n = smt.Int("n")
Nat = smt.Lambda([n], n >= 0)
incr = tdefine("incr", [(n, Nat)], Nat, n + 1)
incr.subtype_lemma
incr.subtype
incr.tele

def lookup(ctx : Telescope, v : smt.ExprRef) -> Optional[tuple[smt.ExprRef, smt.BoolRef]]:
    for v in ctx:
        if isinstance(v, tuple):
            (var, T) = v
            if var.eq(v):
                return (var, T)
        elif v.eq(v):
            return 

def check(ctx : Telescope, t : smt.ExprRef, T : SubSort) -> kd.Proof:
    S,pf = synth(ctx, t)
    if S.eq(T):
        return pf
    else:
        #kd.prove(smt.SubSet(S, T), by=[pf])
        return kd.prove(TForAll(ctx, T(t)), by=[pf])  # unfold=1


def synth(ctx : Telescope, t : smt.ExprRef) -> tuple[SubSort, kd.Proof]:
    """
    Check that an expression has a type in a telescope.
    """
    ctx = norm_tele(ctx)
    for (var, T) in ctx:
        if var.eq(t):
            return T, kd.prove(TForAll(ctx, T(var)))
    if smt.is_app(t):
        decl, children = t.decl(), t.children()
        if decl in subtypes:
            subtype_defn = subtypes[decl]
            T = subtype_defn.T
            subproofs = [check(ctx, c, T1) for c, (v, T1) in zip(children, subtype_defn.ctx)]
            return T, kd.prove(TForAll(subtype_defn.ctx, T(decl(*children))), by=subproofs + [subtype_defn.ax])
    else:
        raise TypeError(f"Could not synthesize type for {t} in {tele}")

synth([(n, Nat)], incr(incr(n) + 1))


TypeError: cannot unpack non-iterable NoneType object

## qforall axioms
```
procedure test(x: Seq, y: Seq) returns (res: Seq)
requires NonEmpty(x) /\ NonEmpty(y);
ensures NonEmpty(res) {
  res := Append(x,y);
}
```

Boogie has intrinsic requires ensures stuff. Hmm.



Actually, should QForAll always / sometimes traverse the P term and get the well formedness conditions too?
QForAll([xs], precond, P)

TForAll([xs], T, P) #?
TForAll([xs], t, T) #?  HasType(xs, t, T)
WellFormed(ctx, pre, t)


That one partial function approach from nipkow reference suggested doing it at `==`

I feel like there is an assertion mode and checkingt mode? In one or the other

return tuple, vs return Anded Version inside quantifier.
def QForAll():
    objligations = wf(P)
    if len(obligations) == 0:
        return QForAll0
    return QForAll0([], cond, P) , QForAll([], cond, obl)
    vs return QForAll([], cond, And(P, oblig))
    vs return QForAll([], And(cond, oblig), P)

Quotients. Subsets ofg powerset stable wrt eq relation.
Set(IntSort()) |  lam A, forall x y, A[x], eq(x,y) => A[y]
def Quot(eq):
    T = eq.domain(0)
    S = SetSort(T)
    A = FreshConst(S)
    return smt.Lambda([A], QForAll([x,y], A[x], eq(x,y), A[y]))
Not higher order though.
def Quot(T : SubSet, eq):
    S = SetSort(T.domain())
    return smt.Lambda([A], QForAll([x,y], T[x], T[y], eq(x,y), A[x], A[y])
Hmm.



It is natural if I have QLambda and QForAll and QExists to also have qdefine.

Could match for QLambda . maybe associate the undefined false branch with the lambda.

if t == Lambda(, If(cond, , undef)):
    
```
# Just as a saniuty check
def QLambda(vs, *args, post=None)
    l = 
    if post 
    l.pf = kd.prove(QForAll( vs, cond, post())
```

This starts to feel more like Dafny.

If proof fails, explanation method could try to traverse don and see which preconditions fail.

From the subset typing persepctive. Dependent types are a methodology for deraling with partiality.

But so is precondition postcondition style reasoning.

What would be the analog of Q reasoning for other models of DTT?
Setoid Specs? Maybe a hoare logic style relational thing.
forall x0 y0, eq(x0,y0), => eq(f(x0),f(y0))
To always show representation independence

n = smt.Int("n")
[(n, Mod3)]
Obviously I can't have n == n % 3, because n is interpreted.

Or to show High Low security noninterference security properties
idefine([(n, High), (m, Low)], body,  Low)
Hmm.

or parametricity
DeclareSort("T")
TypeVar("T") ??? Uhh.
[(n, T), (m, S)]


The curried vs multiairty is a good analogy.
MultiArity Q form is like taking in a refinement tuple (x,y,z) : R(x,y,z)


QInductive is InductiveRel?


def QInductive():
    def create():
        dt = old_create()
        for i in d.num_constructors()
        fun_specs[d.constructor(i), kd.smt.QForAll(  )]






In [ ]:
class FunSpec():
    decl : smt.FuncDeclRef
    #args : list[smt.ExprRef]
    #precond : smt.BoolRef
    #postcond : smt.BoolRef
    #bod
    thm : kd.Proof

fun_specs = {}

def qdefine(name, args, body, pre=None, post=None): # could make pre and post keyword args
    undef = smt.FreshFunction(*[v.sort() for v in args], body.sort())
    f = kd.define(name,   ,smt.If(precond, body, undef(*args)))
    kd.prove(smt.QForAll(args, precond, postcond), by=[body.defn] + 

def FunctionAxiom(name, *sorts, pre=None, post=None, by=[]): # Method ?
    f = smt.Function(name, *sorts)
    f.wf = kd.axiom(smt.QForAll(args, precond, postcond), by=[body.defn] + by)
    fun_specs[f] = FunSpec(f, f.wf)

Ob = smt.DeclareSort("Ob")
Arr = smt.DeclareSort("Arr")
Hom = smt.Function("Hom", Ob, Ob, smt.SetSort(Arr))
id_ = Function("id", Ob, Arr)
id_.wf = kd.axiom(kd.QForAll([a],Hom(a,a)(id_(a))))
fun_specs[id_] = FunSpec(id_, id_.wf)


comp = Function("comp", Arr, Arr, Arr, Arr)
comp.wf = kd.axiom(kd.QForAll([a,b,c,f,g], smt.Implies(smt.And(Hom(a,b)(f), Hom(b,c)(g)), Hom(a,c)(comp(f,g)))))
fun_specs[comp] = FunSpec(comp, comp.wf)

def lemmas(t):
    todo = [t]

def well_formed(vs, pre, t): # "synth"
    if t.decl() in fun_specs:
        spec = fun_specs[t.decl()]
        return kd.prove(smt.QForAll(vs, pre, spec.thm), by=[t] + lemmas(t))
    else:
        return kd.prove(smt.QForAll(vs, pre, smt.BoolVal(True)))

def qprove(vs, pre, P, by=[]): # "check" ?
    vs1, P1 = kd.kernel.herb(smt.QForAll(vs, pre, P))
    typ_lemmas

def qeq(vs, pre, t1, t2):
    lemmas = well_formed(vs, pre, t1)
    lemmas = well_formed(vs, pre, t2)


def wf_lemmas(t): #empty context
    res = []
    todo = [t]
    while todo:
        t = todo.pop()
        decl,children = t.decl(), t.children()
        if decl in fun_specs:
            spec = fun_specs[decl]
            res.append(spec.thm(*children))
            for c in children:
                res.extend(wf_lemmas(c))
    return res
        





In [ ]:
from kdrag.all import *
from kdrag.notation import Telescope, SubSort, TForAll, TExists
from typing import Optional
from dataclasses import dataclass

type QCtx = tuple[list[smt.ExprRef], list[smt.BoolRef]]
@dataclass
class SubTypeDefn():
    decl : smt.FuncDeclRef
    ctx : QCtx
    T : smt.QuantifierRef | smt.ArrayRef  # | smt.FuncDeclRef ?
    ax : kd.Proof # forall xs, precond, prop


def tele_to_qctx(tele : Telescope) -> QCtx:
    """
    Extract the condition from a telescope.
    """
    vs = []
    pred = []
    for v in tele:
        if isinstance(v, tuple):
            (v, T) = v
            vs.append(v)
            if T.sort() == smt.BoolSort():
                pred.append(T)
            elif isinstance(T, smt.ArrayRef) or (isinstance(T, smt.QuantifierRef) and T.is_lambda()):
                pred.append(T(v))
            else:
                raise TypeError(f"Unsupported type for quantifier: {T}")
        elif smt.is_const(v):
            vs.append(v)
            pred.append(smt.BoolVal(True))
        else:
            raise TypeError(f"Unsupported type for quantifier: {v}")
    return (vs, pred)

subtypes = {}

# allow synthesis of output Sort?
def tdefine(name, args : Telescope, T : SubSort, body, by=[]):
    """
    Define a subtype with a name.
    """
    vs,pred = tele_to_qctx(args)
    body_lemma = kd.prove(kd.QForAll(vs, *pred, T(body)), by=by)
    undef = smt.FreshFunction(*[v.sort() for v in vs], body.sort())
    f = kd.define(name, vs, smt.If(smt.And(pred), body, undef(*vs)))
    f.qctx = (vs, pred)
    f.subtype = T
    f.subtype_lemma = kd.prove(kd.QForAll(vs, *pred, T(f(*vs))), by=[body_lemma, f.defn]) # I could do this more directly. It follows from a single unfold.
    subtypes[f] = SubTypeDefn(f, (vs,pred), T, f.subtype_lemma)
    return f

n = smt.Int("n")
Nat = smt.Lambda([n], n >= 0)
incr = tdefine("incr", [(n, Nat)], Nat, n + 1)
incr.subtype_lemma
#incr.subtype
#incr.qctx


def has_type(ctx : Telescope, t : smt.ExprRef, T : SubSort, by=[], **kwargs):
    vs,pred = tele_to_qctx(ctx)
    def doit(vs1):
        todo = [smt.substitute(t, zip(vs1))]
        lemmas = []
        while todo:
            if smt.is_app(t):
                decl, children = t.decl(), t.children()
                if decl in subtypes:
                    subtype_defn = subtypes[decl]
                    lemmas.append(subtype_defn.ax(*children))
                    todo.extend(children)
        return lemmas
    return kd.prove(kd.QForAll(vs, pred, T(t)), by=by, instans=doit  ,**kwargs)




|- ForAll(n, Implies(n >= 0, incr(n) >= 0))

# quote

In [11]:
def quote_sort(s : smt.SortRef):
    T = smt.DeclareSort("'" + s.sexpr())
    return T
def quote(t : smt.ExprRef):
    assert not isinstance(t, smt.QuantifierRef) # we could handle this though.
    T = quote_sort(t.sort())
    decl = t.decl()
    children = [quote(c) for c in t.children()]
    Targs = [c.sort() for c in children]
    absdecl = smt.Function(decl.name(), *Targs, T)
    qt = absdecl(*children)
    qt.unquote = t

    interp = smt.Function("unquote", qt.sort(), t.sort())
    qt.ax = kd.axiom(interp(qt) == t)

    return qt

x + y * z
quote(x+y*z).unquote

x + y*z

In [ ]:
def reify(t : smt.ExprRef):
    # maybe it should be a python value coming in? And use kdrag.reflect.reify?
    assert smt.is_value(t)
    qt = quote(t)
    reify_decl = smt.Function("reify", t.sort(), tq.sort())
    return kd.axiom(reify_decl(t) == qt)

def unquote_reify(t):
    kd.axiom(reify(reflect(t)) == t)


In [ ]:
unquote_sort = {}

"""
def unquote_sort(T : smt.SortRef):
    name = T.name()
    assert name.startswith("'")
    name = name[1:]  # remove leading quote
    if name.startswith("'"):
        return smt.DeclareSort(name)
    elif name == "Real":
        return smt.RealSort()
    elif name == "Int":
        return smt.IntSort()
    elif name == "Bool":
        return smt.BoolSort()
    else:
        raise ValueError(f"Unimplemented sort name: {name}")    
"""
def interp_ax(s : smt.SortRef):
    T = quote_sort(s)
    interp = smt.Function("interp", T, s) 
    reify = smt.Function("reify", s, T)
    x = smt.Const("x", s)
    return kd.axiom(smt.ForAll([x], interp(reify(x)) == x))

def interp_ax(e):
    qe = quote(e)
    interp = smt.Function("unquote", e.sort(), unquote[e.sort()].sort())
    return kd.axiom(interp(qe) == e)
unquote = {}  # interp_table?
# could save away mapping from quoted to unquoted. But then can't interpret anything new made?
# But if I'm actually metaprogramming, just do it in python. This is for internalization.
def quote_sort(s : smt.SortRef):
    T = smt.DeclareSort("'" + s.sexpr())
    #unquote_sort[T] = s
    return T
def quote(t : smt.ExprRef):
    assert not isinstance(t, smt.QuantifierRef)
    T = quote_sort(t.sort())
    decl, children = t.decl(), t.children()
    Targs = [quote_sort(c.sort()) for c in children]
    absdecl = smt.Function(decl.name(), *Targs, T)
    t1 = absdecl(*[quote(c) for c in children])
    interp = smt.Function("unquote", t1.sort(), t.sort())
    t1.ax = kd.axiom(interp(t1) == t)
    t1.unquote = t
    #unquote[t1] = t
    return t1

x + y * z
quote(x+y*z).sort()

#def interp(t : smt.ExprRef):

is_true = smt.Function("is_sat", quote_sort(smt.BoolSort()), smt.BoolSort())

def is_true(t : smt.ExprRef, by=[]):
    # is_true is negation is unsat.
    #s = smt.Solver()
    #s.add(smt.Not(interp(t)))
    #if s.check() == smt.sat:
    #    return kd.axiom(is_true(t) == False, reasons=[])
    p = kd.prove(interp(t), by=by)
    return kd.axiom(is_true(t) == True, reasons=[p])
    

unquote[quote(x+y*z)]



x + y*z

# smt parse
Take all keywords, mangle them.
Use regular parser
turn 
sexp parse

hmm. doesn't actually work.

cmds = [
    smt.Function("declare-fun", name, args, )
]


define_fun


# sequent extract



In [ ]:
from dataclass import dataclass
@dataclass
class Sequent():
    hyps
    concs
def ProofTree():
    judgement : Sequent
    rule : object
    children : list['ProofTree']



def prove(e : smt.ExprRef):
    
    

# define_fun

Maybe doing this could help me understand those early chapters in intro hott

A primitve axiom style?


Like does induction over Nat really require dependent type in some sense?
Also acl2 like ?


def_thm(eqs)
    # define_fun (which deomnstrates termination)
    # but then show it is always True using apporpriate induction principle


https://www.cse.chalmers.se/~ulfn/papers/thesis.pdf
inaccessible patterns

Coquand 92 https://wonks.github.io/type-theory-reading-group/papers/proc92-coquand.pdf
Guguen 2006 https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/99.pdf  Eliminating Dependent Pattern Matching
Mcbride 1999 https://ncatlab.org/nlab/files/conor-mcbride-thesis.pdf
View from Left paper relevant?

Hmm. Dependent pattern matching. indices become unification problems
Case splitting tree. And finding the accessor patterns of the original guy. THis is like my implementation of Pattern match. But I suppose you could go into recursors. Hmm.
Telescopes as tries makes sense because I was distinguishing by value. Hmm.
Individual equations vs all equations

Sozeau


https://jesper.sikanda.be/files/pattern-matching-without-K.pdf pattern macthing wiuthout K. Jesper Cockx




In [ ]:


import kdrag.theories.list as lst
IntList = lst.List(smt.IntSort())
a,b,c = smt.Consts("a b c", IntList)
#rev = smt.Function("rev", IntList, IntList)
#define_fun([rev(IntList.Nil) == IntList.Nil,
#            rev(lst.Cons(a, lst.Cons(b, c))) == lst.Cons(rev(lst.Cons(b, c)), a)
#            ])
x = smt.Int("x")
append = smt.Function("append", IntList, IntList, IntList)
append = define_fun([append(IntList.Nil, b) == b,
            append(lst.Cons(x, c), b) == lst.Cons(x, append(c, b))
            ])
append.case0
append.case1

       add(c!319, c!320) ==
       If(And(is(Z, c!319)),
          c!320,
          If(And(is(S, c!319)),
             S(add(pred(c!319), c!320)),
             unreachable!321))) to ForAll([c!342, c!343],
       add(c!342, c!343) ==
       If(And(is(Z, c!342)),
          c!343,
          If(And(is(S, c!342)),
             S(add(pred(c!342), c!343)),
             unreachable!344)))
       append(c!330, c!331) ==
       If(And(is(Nil, c!330)),
          c!331,
          If(And(is(Cons, c!330)),
             Cons(head(c!330), append(tail(c!330), c!331)),
             unreachable!332))) to ForAll([c!353, c!354],
       append(c!353, c!354) ==
       If(And(is(Nil, c!353)),
          c!354,
          If(And(is(Cons, c!353)),
             Cons(head(c!353), append(tail(c!353), c!354)),
             unreachable!355)))


|- ForAll([c, x, b],
       append(Cons(x, c), b) == Cons(x, append(c, b)))

# partial / extensible


Open predicates gives me internalized prolog.
Could this be useful for the things typeclasses are used for? Automated search?
Extensible datatypes + open functions does help. The integer id is a witness in the 
https://ocaml.org/manual/5.3/extensiblevariants.html

Open Predicates
Open Functions
Open datatypes
https://www.cs.ox.ac.uk/people/ralf.hinze/talks/Open.pdf
https://www.cs.ox.ac.uk/ralf.hinze/publications/PPDP06.pdf
best fit matching... hmm.

Open patterns.

https://srfi.schemers.org/srfi-262/srfi-262.html#examples pattern matyching. Extensible pattern macthing / views. Hmm.
https://arxiv.org/abs/1106.2578  Extensible Pattern Matching in an Extensible Language. Tobin
Universes are open types.
Views was kind of an interesting angle on isomorphism. Snoc Cons views. Different reps of nats, etc. Quotienting.
"The original proposal for extensible pattern matching is Wadler’s ”views” [25], presented in the context of Haskell. In this system, a view is a pair of an injection and projection from an abstract type to an algebraic datatype. Views are intended to support data abstraction in conjunction with pattern matching."

https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/active-patterns active patterns in F# https://en.wikibooks.org/wiki/F_Sharp_Programming/Active_Patterns  https://stackoverflow.com/questions/56536157/what-is-the-main-point-of-active-patterns https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/p29-syme.pdf Extensible Pattern Matching Via a Lightweight Language Extension https://github.com/fsharp/fslang-suggestions/issues/968#issuecomment-764636595 https://github.com/dsyme/dsyme-presentations/blob/master/design-notes/notes-on-learning-fsharp-2.md
Kind of reminds me of TypeIs type guards.

To what degree can actual OO features be applied to a "logical" (pure?) system. There are coinductives. Subtyping. Inheritance. I think these are so confusing that boltiong them on the kernel is fraught with peril, but they can be compiled metafeatures.

https://ghc.gitlab.haskell.org/ghc/doc/users_guide/exts/view_patterns.html 

Maybe views could work as a mechanism.  T <=> {head : int, tail : T} is kind of a view 
section vs retract.  f(g(x)) ~ x. Which equality ?  I guess ~ could be a permutation or something?

I could maybe bolt views into user propagators.
OTT is about making views invisible (?) Deriving the needed casts/coercions.
call 
box(unbox(x))
force(thunk(x))
Lambda([n], T) won't work.


{head = n, tail = box(view(repeat(n))) }  = view(repeat(n))



unbox(box(x)) = x
box(unbox(x)) =? x.  No probably not. What if you boxed it some other way

```python

views = {}

def define_view(inj, proj, by=[]):
views.append(inj,proj, kd.prove(proj(inj(x)) == x)) #? left or right 

```

https://dl.acm.org/doi/abs/10.1145/3674627 Minikanren extensible patterns


Somehow extesnibility veers into object oriented.

https://dl.acm.org/doi/abs/10.1145/3649836 Persimmon: Nested Family Polymorphism with Extensible Variant Types . NEPLS poster. Interesting object orietned bibliography

https://dl.acm.org/doi/10.1145/1159803.1159836  Extensible programming with first-class cases. extensible records and extensible sums are dual

Familia: unifying interfaces, type classes, and family polymorphism  https://dl.acm.org/doi/10.1145/3133894

virtual types Kresten Krab Thorup. 1997. Genericity in Java with virtual types. https://link.springer.com/chapter/10.1007/BFb0053390

nested inheritance

Mads Torgerson. Virtual types are statically safe.

associated types  Chakravorty 2005

Object-Oriented Programming in the Beta Programming Language - Madsen https://en.wikipedia.org/wiki/BETA_(programming_language) https://beta.cs.au.dk/ When why and how of beta https://dl.acm.org/doi/10.1145/1238844.1238854

https://link.springer.com/chapter/10.1007/3-540-45337-7_17 ernst 2001  Family Polymorphism

Scalable extensibility via nested inheritance https://dl.acm.org/doi/10.1145/1028976.1028986

Cardelli badai https://www.amazon.com/Theory-Objects-Monographs-Computer-Science/dp/0387947752 1996 Theory of objects

https://mitpress.mit.edu/9780262526326/theoretical-aspects-of-object-oriented-programming/ Theoretical Aspects of Object-Oriented Programming
Types, Semantics, and Language Design - mitchell and Gunter

could seal to create an actual function definition. Later seals create a different but related definition.
Likewise could seal the cases... Hmm. So a global program compilation kind of thing. We can fullly defunctionalize if we do so.
The results of the different seals have to be ocnsistent, so we could have an axiom:
myfun_ver1(x) == y => myfun(inj(x)) == y





```python
class ExtensibleInductive():
    current : int = 0
    def __init__(self, name, *sorts):
        self.T = self.Record(name, ("index", smt.IntSort()), *sorts)
    def declare(self, name):


Extsenbile("Stream", ("head", IntSort()), ("tail", smt.Array( , ?))


class ExtensibleFun(): # OpenFunction
    decl :
    args : 
    cases = []
    def __init__(self, *args):
        decl = smt.Function(name, smt.IntSort(), *args) # add tag
    def extend(self, cond, body)
    def extend(self, body):
        kd.define(name + "case" + len(cases), args, smt.If(ind == len(cases), body, nextcase(*args))

    # I could narrow the arguments, but not change them?


# maybe code would want more data? You want to scope where the codes came from
class Code():
    current : int
    decode = pdefine("decode")



```





Override vs underride semantics vs enforce disjointness.
It is weird and probably bad 

I have stale axiom around.

Is the undefined version becoming more defined too?

Hmm. COnsidered as a systematic way of making new undef function. Interesting.
foo = if(c, e1, foo_undef1)

It's kind of just a way to invert definition ordering then. Is this even useful?

foo_undef1 = yada.

Oh yeah and the unfolding mechanism still works. Huh.

But for condictuvies or sets it will almost never be able to push through the conditionals. When are sets _not_ equal.


open predicates like prolog. We can only prove True because of outer Or
Or(
    case1, case2, undef_case
)

How to make minikanren extensible like prolog. Need open functions.
Re opening the clark completion form.

Yes, how do you ever know something isn't in predicate? Seal off. Observations?
undef == False would seal off forever. Hmm.
partial seal off?

```python
class OpenPred():
    pred : smt.FuncDeclRef
    args : list[smt.ExprRef]
    cases : list[smt.BoolRef]
    _undef : smt.FuncDeclRef
    #ax : kd.Proof

    def extend(self, body):
        new_undef = smt.FreshFunction()
        kd.define(self.undef.name(), args, smt.Or(body, newundef(*args)))
        self.undef = new_undef

    def __call__(self, *args):
        return self.pred(*args)
```

Hmm. Is this superior to closed function form using If?
Or(Implies(cond, f(args)  == body1)
   Implies(cond2, f(args) == body2)
   Implies(cond3, f(args) == )
)
Or form can start allowing f to have multiple models if clauses overlap. Will we be able to ever prove anything? No I guess.

vs

And(Implies(cond, f(args)  == body1)
   Implies(cond2, f(args) == body2)
   Implies(cond3, f(args) == )
)
And form can have insistency if cond overlap
Makes unfolding hard / impossible.


elem(x,A) ==

pdefine("elem",[x,A], A == pair(y,z), Or(x == y, x == z))

An open set of equations seems reasonable
e

pdefine("head", [s],  s == repeat(n), n)
pdefine("tail", [s],  s == repeat(n), repeat(n))


but it's hard to prove that streams are or aren't equal to other streams.

head(repeate(n), n) == True



In [ ]:
def pdefine(name, args, cond, body):
    sorts = [arg.sort() for arg in args] + [body.sort()]
    f = smt.Function(name, *sorts)
    undef = smt.FreshFunction(name + "_undef", *sorts)
    newbody = smt.If(cond, body, undef(*args))
    if f not in kd.kernel.defns:
        return kd.define(name, args, newbody)
    else:
        defn = kd.kernel.defns[f]
        #kd.prove(smt.Not(smt.And(cond, defn.defined))) # disjoint
        #newbody = smt.substitute(newbody, *zip(args, defn.args))
        old_undef = kd.define(defn.undef.name(), args, newbody)
        newbody = smt.substitute(defn.body, (defn.undef, newbody))
        defn._replace(body=smt.If(cond, body, defn.body))
        # we don't _have_ to replace ax.
        ax = kd.prove(smt.ForAll(args, f(*args) == newbody, by=[defn.ax, old_undef.defn]))
        f.defn = ax
        return f



# termination

Make a measure checking thing.
structural recursion




In [ ]:
#def lex_less(subargs, args):
#    all( subargs[i] <= args[i] for i in range(len(subargs) - 1) ) and subargs[-1] < args[-1]

def arg_dec(cases):
    N = len(cases[0][0])
    for i in range(N):
        if all(kd.utils.is_strict_subterm(subargs[i], args[i]) for args,subargs in cases):
            return i # ith argument is decreasing in all cases.
    return False
    #patterns = [[kd.util.is_subterm(x,y) for x,y zip(arg,subargs)] for args,subargs in cases]
    #for perm in itertools.permutations(n):
    #    if all(   )

#(False, True) < (True, True, True)




SyntaxError: invalid syntax. Perhaps you forgot a comma? (880595545.py, line 5)

In [ ]:
def lex_dec(vs, lhs, rhs):
    if lhs.sort() == smt.IntSort():
        kd.prove(smt.ForAll(vs, 0 <= lhs))
        kd.prove(smt.ForAll(vs, lhs >= rhs))
    else:
        raise ValueError("Unsupported sort for lexicographic decision procedure")
x,y = smt.Ints("x y")
lex_dec([x], )

# for a kd.define, we need to know every subcall is less than original.
# 


# copattern

Extensible definitions
Rewrite rules in simp?


I guess a copattern is a simpler thing than full rewriting


I guess non-deep matching is as simple

add2(S(n)) = 

patterns[add2][S] = ([n], S(S(n)))

https://wiki.c2.com/?ClosuresAndObjectsAreEquivalent
objects and messages
Message = Head | Tail
Response = Int(n) | Stream(s)
repeat(n)[Head] = 

define("repeat", n, Lambda([Msg], If(msg == Head, n, msg==Tail, repeat(n))))

A GADT could maybe make this work
  Head : Msg Int
  Tail : Msg Stream
type Stream = Msg 'a -> 'a

{
    a : T # Code type?
    head : T -> int
    tail : T -> T
}

head = lambda x, If(stream(n) == x, n, undefined)
But you can't extract n out of stream.





In [ ]:
coppatterns = {}
coppatterns[head][repeat] = ([n], n)
coppatterns[tail][repeat] = ([n], repeat(n))




def copattern(pat, rhs, t):
    assert pat.decl() == t.decl()
    zip(pat.children(), t.chidren())

In [ ]:
constructors = {}

# honestly it might be better to do things this way. Instead of searching through junk. I dunno.
List = smt.DeclareSort()
nil = smt.Const("nil", List)
cons = smt.Function("cons", smt.IntSort(), List, List)

constructors[List] = [nil, cons]

In [ ]:
observations = {}

Stream = smt.DeclareSort("Stream")
head = smt.Function("head", Stream, smt.IntSort())
tail = smt.Function("tail", Stream, Stream)
observations[Stream] = [head, tail]

zeros = smt.Const("zeros", Stream)

codefns = {}
def codefine(name, args, obs): # codefine is more like define_fun than it is like define.
    T = obs[0].arg(0).arg(0).sort()
    decl = smt.Function(name,  [arg.sort() for arg in args] ,T)
    assert T in observations
    axs = []
    for n, obeq in enumerate(obs):
        assert smt.is_eq(obeq)
        lhs,rhs = obeq.children()
        ob = lhs.decl()
        assert ob == observations[T][n]
        axs.append(smt.ForAll(args, ob))
        codefns[(decl, ob)] = (vs, rhs)
    return axs


zeros = codefine("zeros", [],
         [head(zeros) == 0,
          tail(zeros) == zeros])

wrap = smt.Function("wrap", StreamT, Stream)
codefine("zeros", wrap(StreamT(0, zeros)))


#codefine

# solver for

Hmmm. Wait. This is a guassian eliminator?




In [68]:
import z3
def solve(eqs, x):
   s = z3.SimpleSolver()
   s.add(eqs)
   return s.solve_for([x])
x,y,z = z3.Reals('x y z')
solve([x + y == z], x)

[]

In [9]:
import z3
x,y,z = z3.Reals('x y z')
s = z3.Solver()
s.add(x + y == z)
print(z3.get_version_string())
print(s.solve_for([x]))
#s.solve_for1(x)


4.15.0
[]


In [9]:
z3.IntVal(3).py_value()

3

In [7]:
z3.get_version_string()

'4.15.0'

In [27]:
import z3
x,y,z,p = z3.Reals('x y z p')
s = z3.SimpleSolver()
s.add(x + y == z)
s.add(p + y + 5 == z)
s.check()
print(s.solve_for([x]))

[(z, p + y + 5, And(y + -1*z + p <= -5, y + -1*z + p >= -5)), (x, z + -1*y, And(x + y + -1*z >= 0, x + y + -1*z <= 0))]


https://github.com/Z3Prover/doc/tree/master/synthesiz3
https://github.com/Z3Prover/doc/commit/690d88ba43a7d706d174da2c48279a5d16810a92

# Proof
z3 proof objects were an ast.
They even had modus ponens?
Hmm.
Look at z3 and cvc5 proof objects

https://ceur-ws.org/Vol-3185/paper9527.pdf simple proof format for SMT
https://ceur-ws.org/Vol-3455/invited1.pdf Challenges in SMT Proof Production and Checking for
Arithmetic Reasoning

In [103]:
from kdrag.all import *
s = smt.Solver()
s.set("clause_proof", True)
f = smt.Function("f", smt.IntSort(), smt.BoolSort())
s.add(smt.ForAll([x], f(x)))
s.add(smt.Not(f(1)))
print(s.check())
p = s.proof()
p.decl()
assert not p.sort() == smt.DeclareSort("Proof")
p.sexpr()

unsat


'(let ((a!1 (forall ((x Int)) (! (f x) :pattern ((f x))))))\n(let ((a!2 (assumption (inst a!1 (not (f 1)) (bind 1) (gen 0))\n                       (or (not a!1) (f 1)))))\n  (proof-trail (assumption assumption a!1)\n               (assumption assumption (not (f 1)))\n               a!2\n               false)))'

In [ ]:
Proof = smt.DeclareSort("Proof")
andI = smt.Function("andI", Proof, Proof, Proof) # to build multicontext
prove = smt.Function("prove", smt.BoolSort(), Proof, Proof)
axiom = smt.Function("axiom", smt.BoolSort(), Proof)

#instan = smt.Function("instan", Proof, Proof)

# both need to be multiarity / parametric
#T = smt.TypeVar("T")
#herb = smt.Function("herb", smt.ArraySort(), Proof)
#instan = smt.Function("instan", Proof, Proof)





# cvxpy

But really, doesn't z3 offer this? It's about scale.
I guess it doesn't offer SDP
We might also want to verify cvxpy transforms like that lean project?

duality? 

In [ ]:
import cvxpy as cp
from kdrag.all import *
import operator

x,y = smt.Reals("x y")
binop = {
    (x <= y).decl() : operator.le,
    (x >= y).decl() : operator.ge,
    (x == y).decl() : operator.eq,
}



def tocvxpy(e : smt.ExprRef):
    if isinstance(e, smt.BoolRef):
        if e.decl() in binop:
            return binop[e.decl()](tocvxpy(e.arg(0)), tocvxpy(e.arg(1)))
        else:
            return 
        return cvxpy.Constant(e.py_value())

x = cp.Variable(1, name="x")
x


Variable((1,), x)

https://en.wikipedia.org/wiki/Sum-of-squares_optimization

In [4]:
import z3
x,y = smt.Reals("x y")
z3.prove(x**2 - 4*x*y + 7*y**2 >= 0)


proved


# sympy
We're stuck. Just let soundness go

Also tying together a solver could be fun

taylor series





In [ ]:
import kdrag as kd
import kdrag.theories.real as real
from kdrag.theories.real.sympy import sympify, simplify, solve, kdify
from kdrag.all import *


x,y,z = smt.Reals("x y z")
sympify(x + y == z)
sympify(x + y <= z)
#implify(real.cos(x)**2 + real.sin(x)**2 == z)
simplify(real.cos(x) == 3)

solve([real.cos(x) == 3], [x])
#solve([real.exp(x) == x], [x])

t = sympify(real.deriv(smt.Lambda([x], real.cos(real.sin(x)))))

simplify(real.integrate(smt.Lambda([x], real.sin(x)), 0,x))


sympify(real.summation(smt.Lambda([x], real.exp(-x)), 0, y)) # hmm
simplify(real.deriv(smt.Lambda([x], real.cos(real.sin(x)))))
import sympy
a = sympy.Symbol('a')
kdify(sympy.Lambda(a,a))
simplify(smt.Lambda([x], 2 + x + 1))

class Lemma(kd.Lemma):
    def simp(self, at=None, unfold=False):
        super().simp(at=at, unfold=unfold)
        goalctx = self.top_goal()
        goal = goalctx.goal
        if at is None:
            newgoal = simplify(goal)
            print(newgoal)
            self.lemmas.append(kd.axiom(goal == newgoal, by=["sympy simplify"]))
            self.goals[-1] = goalctx._replace(goal=newgoal)
        return self.top_goal()
    def solve(self, vs):
        goalctx = self.goals.top_goal()
        return solve(goalctx.ctx, vs)
    #def solve_for(self, vs):
    #    contraints = self.goals[-1].ctx
    #    s = smt.SimpleSolver()
    #    s.add(contraints)
    #    return s.solve_for(vs)

l = Lemma(real.cos(x)**2 + real.sin(x)**2 == 1)
l.simp()
l.qed()




True


|- cos(x)**2 + sin(x)**2 == 1

In [ ]:
(smt.Lambda([x], x))

SympifyError: SympifyError: "cannot sympify object of type <class 'kdrag.reflect.KnuckleClosure'>"

In [7]:
(-x).decl().kind()

520

In [2]:
dir(smt.Lambda([x], x))
smt.Lambda(x,x)

Lambda(x, x)

In [10]:
import sympy
a = sympy.Symbol('a')
a.name
def Lambda(x,y): return x,y
#sympy.lambdify([], sympy.Lambda(a, a), modules={"Lambda" : Lambda})()
sympy.E
isinstance(sympy.Lambda(a,2+a), sympy.Lambda)
sympy.Lambda(a,2+a).variables

(a,)

In [8]:
sympy.Lambda(a, a + 7 * a).atoms()

{8, a}

In [2]:
a

NameError: name 'a' is not defined

# eexists / uvars

uvars realkly belong to the total goal state not Goals.
tracking what is allowed in them uses the goal vars at the time.
Yikeys.
Can just collect junk to be included in prove(smt.Exists(uvars, ))

prove(smt.Exist)

kd.QExists(uvars, constraints, whatev)

Kind of need uvars to use solve_for and sympy stuff?


In [ ]:
class Lemma():
    uvars : set[smt.ExprRef]
    constraints : list[smt.BoolRef] # or go into lemmas?
    



def eexists()

# spacer

Can we use spacer for something? Inductive theorem proving?
https://microsoft.github.io/z3guide/programming/Z3%20Python%20-%20Readonly/Fixedpoints
https://github.com/Z3Prover/z3/discussions/5013
https://github.com/agurfinkel/spacer-on-jupyter/blob/master/Dagstuhl2019.ipynb
https://github.com/agurfinkel/spacer-on-jupyter/blob/master/FSU_2023.ipynb
https://github.com/agurfinkel/spacer-on-jupyter/blob/master/src/spacer_tutorial/solve.py


DIY IC3 solver?
Needs interpolation right?
Could just use spacer for interp

https://github.com/Z3Prover/z3/blob/master/examples/python/mini_ic3.py
https://github.com/pddenhar/Z3-IC3-PDR

https://github.com/arbrad/IC3ref 

https://github.com/JakubSarnik/geyser
https://github.com/gipsyh/rIC3

aiger and btor2



In [14]:
from kdrag.all import *
def solver_horn(chc):
    # https://github.com/agurfinkel/spacer-on-jupyter/blob/master/src/spacer_tutorial/solve.py
    s = z3.SolverFor('HORN')
    s.set('engine', 'spacer')
    s.set('spacer.order_children', 2)
    s.add(chc)
    res = s.check()
    if res == z3.sat:
        return res, s.model()
    elif res == z3.unsat:
        return res, s.proof()
    else:
        return res, None
    
def interpolate(As, A, Bs, B, shared):
    # Uninterpreted predicate Itp over shared symbols
    Itp = z3.Function('Itp', [s.sort() for s in shared] + [z3.BoolSort()])

    # first CHC: A ==> Itp
    left = z3.ForAll([a for a in As], z3.Implies(A, Itp(shared)))
    # second CHC: Itp ==> !B
    right = z3.ForAll([b for b in Bs], z3.Implies(Itp(shared), z3.Not(B)))

    # run CHC solver
    res, answer = solve_horn([left, right])

    if res == z3.sat:
        return answer.eval(Itp(shared))
    else:
        return None





https://microsoft.github.io/z3guide/docs/fixedpoints/engineforpdr

In [ ]:
mc = smt.Function("mc", smt.IntSort(), smt.IntSort(), smt.BoolSort())
n,m,p = smt.Ints("n m p")
q1 = smt.Function("q1", smt.IntSort(), smt.IntSort(), smt.BoolSort())
clauses = [
    kd.QForAll([m], m > 100, mc(m, m - 10)),
    kd.QForAll([m,p,n], m <= 100, mc(m + 11, p), mc(p,n), mc(m,n)),
    #kd.QForAll([m,n], mc(m,n), n < 91, q1(m,n)),
]
solver_horn(clauses)



(sat, [mc = [else -> True], q1 = [else -> True]])

https://z3prover.github.io/papers/programmingz3.html#sec-horn-clause-solver

In [25]:
s = smt.SolverFor("HORN")
Z = smt.IntSort()
B = smt.BoolSort()
mc = smt.Function('mc', Z, Z, B)
x, y, z = smt.Ints('x y z')
s.add(smt.ForAll(x, smt.Implies(x > 100, mc(x, x - 10))))
s.add(smt.ForAll([x, y, z], 
             smt.Implies(smt.And(x <= 100, mc(x + 11, y), mc(y, z)), 
                     mc(x, z))))
s.add(smt.ForAll([x, y], smt.Implies(smt.And(x <= 101, mc(x, y)), y == 91)))
s.add(smt.ForAll([x, y], smt.Implies(smt.And(x >= 101, mc(x, y)), x == y + 10)))
print(s.check())
print(s.model().eval(mc(x, y)))

sat
And(Not(y + -1*x <= -11),
    Not(y <= 90),
    Or(Not(y >= 92), Not(y + -1*x >= -9)))


## popper
https://github.com/logic-and-learning-lab/Popper/

Houdini

Is invariant (?)

def inv(P)
    for P in candidates()
        try:
            kd.prove(induct(P).arg(0))





# tableau / meson

For modal logics might be nice
Also smt boosted meson is interesting in and of itself.



# brute check
For anything 32bit or less we ought to just brute check it.

float16 for example. Would give me more confidence.
single variable float32 ops



# Solvers
##  isabelle
Graham was saying why not just download isabelle... Hmm.



```bash
emconfigure ./configure --enable-ho
emmake make AR="/home/philip/Downloads/emsdk/upstream/emscripten/emar rcs"
```

 And had to comment out in Makefile.vars the forced CC and AR values


https://github.com/philzook58/eprover/releases/download/E.3.2.5-ho/eprover-ho

bitwuzla
cvc5 
ytices
pysmt

bto2
moxi

implement model checking algorithms?



In [ ]:
import pyboolector as btor

btor.Parse("example.btor")



In [ ]:
import urllib.request
import os
import stat
filename = "eprover-ho"
#urllib.request.urlretrieve("https://github.com/philzook58/eprover/releases/download/E.3.2.5-ho/eprover-ho", filename)
st = os.stat(filename)
    # Add execute permissions for user, group, and others
os.chmod(filename, st.st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)

In [77]:
from kdrag.solvers import EProverTHFSolver
s = EProverTHFSolver()
s.add(x == y)
s.check()
s.res.stdout

b'# Preprocessing class: HSSSSMSSSSSNFFN.\n# Scheduled 4 strats onto 8 cores with 300 seconds (2400 total)\n# Starting new_ho_10 with 1500s (5) cores\n# Starting ho_unfolding_6 with 300s (1) cores\n# Starting sh4l with 300s (1) cores\n# Starting ehoh_best_nonlift_rwall with 300s (1) cores\n# new_ho_10 with pid 996928 completed with status 9\n# ho_unfolding_6 with pid 996929 completed with status 9\n# sh4l with pid 996930 completed with status 9\n# ehoh_best_nonlift_rwall with pid 996931 completed with status 9\n# Schedule exhausted\n# SZS status GaveUp\n'

In [75]:
import subprocess
subprocess.run(["./eprover-ho", "--help"], check=True)


E 3.2.5-ho "Puttabong Moondrop"

Usage: eprover [options] [files]

Read a set of first-order (or, in the -ho-version, higher-order)
clauses and formulae and try to prove the conjecture (if given)
or show the set unsatisfiable.

Options:

   -h
  --help
    Print a short description of program usage and options.

   -V
  --version
    Print the version number of the prover. Please include this with all bug
    reports (if any).

   -v
  --verbose[=<arg>]
    Verbose comments on the progress of the program. This differs from the
    output level (below) in that technical information is printed to stderr,
    while the output level determines which logical manipulations of the
    clauses are printed to stdout. The short form or the long form without
    the optional argument is equivalent to --verbose=1.

   -o <arg>
  --output-file=<arg>
    Redirect output into the named file.

   -s
  --silent
    Equivalent to --output-level=0.

   -l <arg>
  --output-level=<arg>
    Select an outpu

CompletedProcess(args=['./eprover-ho', '--help'], returncode=0)

# egraph

Accept prolog and smtlib syntax?
like the good old days



## Mixins vs subclassing

GaussMixin()
t = GaussTheory()
EGraph(GaussTheory(), SympyTheory(), )

self.theories = []

def find(self, x):
    for t in theories:
        t.find(x)

def union():
    todo = []
    for t in theories:
        todo.extend(t.union())


## Lambda
alpha_norm before using add term.

Recursively carry params down through?
Eta long form.

A normal form?

Do slotted. Add some knd of permutation egraph?


eta_maximal form

In [ ]:
def eta_long(ctx, t : smt.ExprRef):
    if isinstance(t, smt.QuantifierRef):
        

## coegraph

Maybe use cvc5 codatypes
Datatypes already achieve the cocongruence propagation



## aptroot
https://mastodon.gamedev.place/@harold/114586279569895001




In [ ]:
from kdrag.all import *
Fruit = kd.Enum("Fruit", ["A", "B"])
import kdrag.theories.seq as seq_

FString = seq_.SeqSort(Fruit)

seq_.SeqVal([Fruit.A]*3) == seq_.SeqVal([Fruit.A, Fruit.B, Fruit.A])





## ILP extract
cvxpy translation?


import extraction gym?



## Gauss

Sympy, solve_for, passaegmath msolve



In [ ]:
class GaussEGraph():
    def __init__(self):
        self.eqs = smt.SimpleSolver() # Or sympy grobner basis.
    def union(self, x, y):
        if isinstance(smt.ArithRef):
            self.eqs

class GrobnerEGraph(EGraph):


In [ ]:

def sympy_simplify(self):
    for t in self.terms:
        if t.sort() == smt.RealSort():



## Extract

In [43]:

from kdrag.all import *
from kdrag.solvers.egraph import EGraph
from collections import defaultdict
def extract(self, t0 : smt.ExprRef, cost_fun = (lambda _: 1)):
    inf = float("inf")
    best_cost = defaultdict(lambda: inf)
    best = {}
    while True:
        done = True
        # Terms are taking the place of enodes.
        for t in self.terms.values():
            eid = self.find(t)
            cost = cost_fun(t) + sum([best_cost[self.find(c)] for c in t.children()]) # cost_fun(t.decl()) ?
            if cost < best_cost[eid]:
                best_cost[eid] = cost
                best[eid] = t
                done = False
        if done:
            break
    #@functools.cache
    def build_best(t):
        t1 = best[self.find(t)]
        return t1.decl()(*[build_best(c) for c in t1.children()])
    return build_best(t0)

E = EGraph()
x,y,z = smt.Ints('x y z')
E.add_term(x + y)
E.rebuild()
assert extract(E, x + y).eq(x + y)
E.union(x + y, y)
E.rebuild()
assert extract(E, x + y).eq(y)




False

## Proof

In [64]:
s = smt.Solver()
s.set(unsat_core=True)
a,b,c,d, e = smt.Ints('a b c d e')
id_map = {}
def tassert(a):
    tid = a.get_id()
    id_map[tid] = a
    #s.assert_and_track(a, smt.Bool(str(a)))
    s.assert_and_track(a, smt.Bool(str(tid)))
def core():
    core = s.unsat_core()
    return [id_map[int(str(c))] for c in core]


tassert(a == b)
tassert(b == c)
tassert(e == d)
#s.add(a == b)
#s.add(b == c)
#s.assert_and_track(a == b)
#s.assert_and_track(b == c, b == c)
s.add(a != c)
s.check()
#s.unsat_core()
type(core()[0])

z3.z3.BoolRef

In [ ]:
class EgraphProof(EGraph):
    def __init__(self):
        super().__init__()
        self.reasons = {}
        self.solver.set("unsat_core", True)
    def union(self, e1, e2, reason=None):
        super().union(e1, e2)
        if reason is not None:
            p = smt.FreshConst(smt.BoolSort())
            self.reasons[p.get_id()] = (e1,e2,reason)
            self.solver.assert_and_track(e1 == e2, p) 
    def add(self, expr, reason=None): # Use this indriectio nin Egrpha, so we can use assert and track
        tid = expr.get_id()
        self.terms[tid] = expr
        p = smt.FreshConst(smt.BoolSort())
        self.reasons[p.get_id()] = reason
        self.solver.assert_and_track(expr, p)
    def core(self):
        core = self.solver.unsat_core()
        return [self.terms[int(str(c))] for c in core]


Return instantiations of rules that make this possible
(vs, rule, lhs = rhs)


return ground rewrite system that'll do it? But then each rule needs to be itself dignified. bleh.

I could also use the trick from the third post of using z3 ematcher and grab the instans

Also it'd be neat to extract those into a reified proof object.


## Context
Egraph does support hypothetical equalities?
But I won't ematch unless I'm in that hypothetical situation

ctxs = []
ctx rebuilding - which ctx subsume, which 
Implies(ctx1, ctx2)
Implies(ctx2, ctx1)  ---> 

contextual rewrite rules.

Brute force contextual egraph

Ok so i can build it.
But then what is it for?
Well, if then else is a good example.

add_term(self, ctx, t):
    if smt.is_ite(t):
        ctx.append(t.arg(0)), t.arg(1)
        ctx.append(smt.Not(t.arg(1))), t.arg(0)



There's a lot of pruning possible in these queries.

In [44]:
from kdrag.all import *

s = smt.Solver()
with s:
    s.add(False)
    print(s.check())
print(s.check())



unsat
sat


In [ ]:
    def rebuild(self):
        super().rebuild()
        ctxs = self.roots[smt.BoolSort()] & self.ctxs
        for ctx1 in ctxs:
            for ctx2 in ctxs:
                if ctx1 != ctx2 and self.subsumes(ctx1, ctx2):
                    self.solver.add(smt.Not(smt.Implies(ctx1, ctx2)))

In [ ]:
from kdrag.all import *
from kdrag.solvers.egraph import EGraph
def ctx_order(ctxs):
    s = smt.Solver()
    eq = []
    le = []
    for ctx1 in ctxs:
        for ctx2 in ctxs:
            s.push()
            s.add(smt.Not(ctx1 == ctx2))
            res = s.check()
            s.pop()
            if res == smt.unsat:
                eq.append((ctx1,ctx2))
            s.push()
            s.add(smt.Not(smt.Implies(ctx1, ctx2)))
            res = s.check()
            s.pop()
            if res == smt.unsat:
                le.append((ctx2,ctx1))

def subsumes(ctx1, ctx2):
    s = smt.Solver()
    s.add(smt.Not(smt.Implies(ctx1, ctx2)))
    return s.check() == smt.unsat
def iff(ctx1, ctx2):
    s = smt.Solver()
    s.add(smt.Not(ctx1 == ctx2))
    return s.check() == smt.unsat            


x,y,z = smt.Ints('x y z')

ctx1 = smt.And(x == y, y == z)
ctx2 = x == z
assert subsumes(ctx1, ctx2)
assert not subsumes(ctx2, ctx1)
iff(ctx1, ctx1)

class CtxEGraph(EGraph):
    ctxs : set[int]
    subsume : dict[int, set[int]] # "path" transitive closure

    # there's two versions. One where you have pattern vars in ctx or not. or if contexrt is pattern.
    # suearch over all subsumed contexts.
    # def ematch_ctx(self, vs, pctx, lhs):
    def ematch_ctx(self, vs, lhs):
        res = []
        for ctx in self.ctxs:
            #self.substitute(ctx, vs)
            with self.solver: # takes the push?
                self.solver.add(self.terms[ctx])
                res.extend((ctx, match_) for match_ in self.ematch(vs, lhs))
        return res
    #def ctx_sub():
    def rebuild(self):
        super().rebuild()
        oldctxs = self.ctxs
        for ctx1 in self.ctxs:
            for ctx2 in self.ctxs:
                if ctx2 in self.subsume[ctx1]:
                    continue
                else:
                    self.solver.push()
                    self.solver.add(smt.Not(smt.Implies(ctx1, ctx2)))
                    self.solver.pop()





In [ ]:
def add_term_quant(term):
    t = alpha_norm(term)
    todo = [([], t)]
    while todo:
        ctx, t = todo.pop()
        



# datalog
z3 has a datalog, but whatev. Take snakelog approach https://github.com/philzook58/snakelog

Also homommorphism finder. A la CSP / sqlite blog post

Pretty print into ASP?

spacer?


sqlite dump of binary
sqlite dump of clang ast or treesitter




In [ ]:
from kdrag.all import *
x,y,z = smt.Ints("x y z")
edge = smt.Function("edge", smt.IntSort(), smt.IntSort(), smt.BoolSort())
path = smt.Function("path", smt.IntSort(), smt.IntSort(), smt.BoolSort())
edgepath = rw.rule_of_expr(kd.QForAll([x,y], edge(x,y), path(x,y)))
trans = rw.rule_of_expr(kd.QForAll([x,y,z], edge(x,y), path(y,z), path(x,z)))


def compile_body(vs : list[smt.ExprRef], body : smt.BoolRef) -> str:
    todo = [body]
    env = {}
    froms = []
    wheres = []
    counter = 0
    while todo:
        rel = todo.pop()
        if smt.is_eq(rel):
            raise ValueError("Equality not supported")
        elif smt.is_not(rel):
            raise ValueError("Negation not supported")
        elif smt.is_or(rel):
            raise ValueError("Disjunction not supported")
        elif smt.is_and(rel):
            todo.extend(rel.children())
        elif smt.is_true(rel):
            continue
        elif smt.is_app(rel):
            name = rel.decl().name()
            args = rel.children()
            row_name = name + str(counter)
            counter += 1 
            froms.append(f"{name} AS {row_name}")
            for n, arg in enumerate(args):
                if arg in vs:
                    if arg in env:
                        wheres.append(f"{env[arg]} = {row_name}.x{n}")
                    else:
                        env[arg] = f"{row_name}.x{n}"
                else:
                    wheres.append(f"{row_name}.x{n} = {str(arg)}")
        else:
            raise ValueError(f"Unsupported expression: {rel}")
    return env, froms, wheres

def compile_rule(rule : kd.rewrite.Rule) -> str:
    env, froms, wheres = compile_body(rule.vs, rule.hyp)
    name = rule.conc.decl().name()
    selects = []
    for n,arg in  enumerate(rule.conc.children()):
        if arg in rule.vs:
            if arg in env:
                selects.append(f"{env[arg]} AS x{n}") # maybe select as keyword
            else:
                raise ValueError(f"Variable {arg} not found in body")
        else:
            selects.append(f"{arg} AS x{n}")
    froms = ", ".join(froms)
    wheres = " AND ".join(wheres)
    selects = ", ".join(selects)
    return f"""
    INSERT OR IGNORE INTO {name} SELECT DISTINCT {selects}
    {"FROM " + froms if froms else ""}
    {"WHERE " + wheres if wheres else ""}
    """

print(compile_rule(edgepath))
compile_rule(kd.rewrite.rule_of_expr(edge(1,2)))



    INSERT OR IGNORE INTO path SELECT DISTINCT edge0.x0 AS x0, edge0.x1 AS x1
    FROM edge AS edge0
    
    


'\n    INSERT OR IGNORE INTO edge SELECT DISTINCT 1 AS x0, 2 AS x1\n    \n    \n    '

In [ ]:
import sqlite3
class Datalog():
    def __init__(self):
        self.db = sqlite3.connect(":memory:")
    
    def declare_sig(self, sig : list[smt.FuncDeclRef]):
        for f in sig:
            primkey = "(" + ", ".join([f"x{i}" for i in range(f.arity())]) + ")"
            self.db.execute(f"""CREATE TABLE IF NOT EXISTS {f.name()} 
                            ({', '.join([f'x{i} {f.range().name()}' for i in range(f.arity())])}, 
                             PRIMARY KEY {primkey})""")

    def run(self, rule : kd.rewrite.Rule | smt.BoolRef):
        if isinstance(rule, smt.BoolRef):
            rule = kd.rewrite.rule_of_expr(rule)
        sql = compile_rule(rule)
        self.db.execute(sql)

s = Datalog()
s.declare_db([edge, path])
s.run(edge(1,2))
s.run(edge(2,3))
s.run(edgepath)
s.run(trans)
s.run(trans)
s.db.execute("SELECT * FROM path").fetchall()
        

[(1, 2), (2, 3), (1, 3)]

# compile out of smt2

Look at basic compiler books.
Tiger
Essentials
nanopass

smtcc :)
prescheme

undefined behavior.


We can make a meta layer list
https://pypy.org/posts/2024/07/toy-abstract-interpretation.html
https://pypy.org/posts/2022/07/toy-optimizer.html

In [ ]:

def ret(x):
    return "mov rdi, " + str(x) + "\n" + \


In [1]:
%%file /tmp/test.c
#include <stdint.h>
uint64_t mytest(){
    return 42;
}


Writing /tmp/test.c


In [3]:
! gcc -S -o /tmp/test.s /tmp/test.c && cat /tmp/test.s

	.file	"test.c"
	.text
	.globl	mytest
	.type	mytest, @function
mytest:
.LFB0:
	.cfi_startproc
	endbr64
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	movl	$42, %eax
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE0:
	.size	mytest, .-mytest
	.ident	"GCC: (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0"
	.section	.note.GNU-stack,"",@progbits
	.section	.note.gnu.property,"a"
	.align 8
	.long	1f - 0f
	.long	4f - 1f
	.long	5
0:
	.string	"GNU"
1:
	.align 8
	.long	0xc0000002
	.long	3f - 2f
2:
	.long	0x3
3:
	.align 8
4:


In [4]:
%%file /tmp/test.S

.globl mytest
mytest:
    mov $42, %rax
    ret


Overwriting /tmp/test.S


In [9]:
!gcc -c -fPIC -o /tmp/test.o /tmp/test.S && ld -shared -o /tmp/test2.so /tmp/test.o && objdump -d /tmp/test.so


/tmp/test.so:     file format elf64-x86-64


Disassembly of section .text:

0000000000001000 <mytest>:
    1000:	48 c7 c0 2a 00 00 00 	mov    $0x2a,%rax
    1007:	c3                   	ret


In [ ]:
import cffi

ffi = cffi.FFI()
ffi.cdef("""uint64_t mytest();""")
lib = ffi.dlopen("/tmp/test2.so")
dir(lib)
print(lib.mytest())
del lib

def 



42


In [ ]:
World = smt.DeclareSort("World")
BV64 = smt.BitVecSort(64)
mem = smt.Function("mem", World, smt.ArraySort(BV64, BV64))

ret = smt.Function("ret", BV64, World, World)


type Block = list[smt.ExprRef]

class Module():
    

#class Block():
#   stmts : list[smt.ExprRef]



In [ ]:
x,y,z = smt.BitVecs("x y z", 64)
getarg = smt.Function("getarg", smt.IntSort(), smt.BitVecSort(64))
copy = smt.Function("copy", smt.BitVecSort(64), smt.BitVecSort(64)) # kd.define("copy", [x], x)
prog = [
    a := x + y,
    b := a + z,
    c := a - b,
]
def address_code(prog):
    seen = set()
    for stmt in prog:
        if all(c in seen for c in stmt.children()):
            seen.add(stmt)
        else:
            return False
    return True
# path assumptions could be interesting.

def esimplify(prog):
    uf = {}
    new_prog = []
    for n,stmt in prog:
        s = smt.Solver()


def register(prog):
    live = set()
    regalloc = {}
    availreg = ["r8", "rsi","rdi","rax"]
    for stmt in reversed(prog):
        availreg.append(regalloc[stmt])
        live.remove(stmt)
        for c in stmt.children():
            if c not in live:
                live.add(c)
                if availreg:
                    regalloc[c] = availreg.pop()
                else:
                    # spill
                    raise Exception("No registers available")

    




[x + y, x + y + z, x + y - (x + y + z)]

In [ ]:
from kdrag.all import *
one = smt.BitVecVal(1, 64)
zero = smt.BitVecVal(0, 64)

one + zero

def to_asm(e : smt.ExprRef):
    if isinstance(e, smt.BitVecNumRef):
        return "mov rdi, %d" % e.as_long()
    elif smt.is_add(e):
        to_asm(e.args[0])
        return [
            f"pop %rdi",
            r"add rdi, %d" % e.args[1].as_long()
        ]
    






1 + 0

# vectorize

pshufb
pext pdep

Aptroot's bot.

These chess guys are nuts
https://www.chessprogramming.org/BMI2#PEXTBitboards 


In [ ]:
# https://jamesbornholt.com/papers/diospyros-asplos21.pdf
BV16 = smt.BitVecSort(16)
BV8 = smt.BitVecSort(8)

x,y,z = smt.BitVecs("x y z", 16)
vadd = smt.Function('vadd', BV16, BV16, BV16)
vsub = smt.Function('vsub', BV16, BV16, BV16)
vmul = smt.Function('vmul', BV16, BV16, BV16)
vneg = smt.Function('vneg', BV16, BV16)
vmac = smt.Function('vmac', BV16, BV16, BV16, BV16)

vadd(x,y) == smt.Concat(smt.Extract(15, 8, x) + smt.Extract(15, 8, y), 
                        smt.Extract(7, 0, x) + smt.Extract(7, 0, y))

vsub(x,y) == smt.Concat(smt.Extract(15, 8, x) - smt.Extract(15, 8, y), 
                        smt.Extract(7, 0, x) - smt.Extract(7, 0, y))
vmul(x,y) == smt.Concat(smt.Extract(15, 8, x) * smt.Extract(15, 8, y),
                        smt.Extract(7, 0, x) * smt.Extract(7, 0, y))
vneg(x) == smt.Concat(-smt.Extract(15, 8, x), -smt.Extract(7, 0, x))
vmac(x,y,z) == smt.Concat(smt.Extract(15, 8, x) * smt.Extract(15, 8, y) + smt.Extract(15, 8, z),
                        smt.Extract(7, 0, x) * smt.Extract(7, 0, y) + smt.Extract(7, 0, z))

smt.prove(smt.Concat(smt.Extract(15, 8, x), smt.Extract(7, 0, x)) == x)

vadd(vmul(x,y),z) == vmac(x,y,z)

# asymptotic

https://github.com/teorth/estimates
https://en.wikipedia.org/wiki/Big_O_notation 
https://www.andrew.cmu.edu/user/avigad/Papers/bigo.pdf  Formalizing O notation in Isabelle/HOL

eqaulity isn't equality (unless it is?)

f(x) = O(g(x)) is really
O(g)[f]

But to do this is to ignore the wisdom of the notation perhaps. because formalists can't fit a notation into their rigid paradgims doesn't mean its bad

I could do custom overloading of == to mean asymptotic.
Didn't I have some idea about O ~ id



nonstandard reals
Kind of a pain in the ass to do it this way. Is the injectivity it offers us even that useful?



In [ ]:
NonR0 = smt.DeclareSort("NonR0")
NonR = kd.Inductive("NonR")
NonR.declare("Std", ("R", smt.RealSort()))
NonR.declare("NonStd", ("Nr", NonR0))
NonR = NonR.create()

# std predicate is now is_Std
x,y = smt.Consts("x y", NonR)
add = kd.notation.add.define([x,y],
                             kd.cond(
                                (smt.And(smt.is_Std(x), smt.is_Std(y)), x + y),
                                (smt.And(smt.is_Std(x), smt.is_NonStd(y)), x + y),
                                (smt.And(smt.is_NonStd(x), smt.is_Std(y)), x + y),
                                (smt.And(smt.is_NonStd(x), smt.is_NonStd(y)), x + y
                             )





# pysmt
I never _really_ gave pysmt a fair shake.
It doesn't support lambda. That is a pretty tough blow



In [31]:
from pysmt.shortcuts import Symbol, And, Not, is_sat

varA = Symbol("A") # Default type is Boolean
varB = Symbol("B")
f = And(varA, Not(varB))
is_sat(f)
g = f.substitute({varB: varA})
is_sat(g)

False

# lambda unify


In [ ]:
def unify_huet():
    todo = []
    frozen = [] #flexflex?
    subst = {}
    while True:
        while todo:
            current = todo.pop()
            # do unification.

        for p in frozen:
            if flexflex(p):
                continue
            else:
                frozen.remove(p)
                todo.append(p)
                break
        else:
            return subst, frozen
    
        

    

# GAT / refinement
see types.ipynb also


The property of being a proper telescoping mapping requires an smt solve?

why bother teelscoping? Isabelle pure has no telescoping effect? CVody says that helps build model but did not elaborate
https://argo.matf.bg.ac.rs/events/2009/fatpa2009/slides/Wenzel_PureLogicalReasoning%20in%20Isabelle-Isar.pdf

https://arxiv.org/pdf/1911.00399 hott in pure
https://arxiv.org/pdf/cs/9301105 pualson foundation of generic theorem prover


So a conditional judgement can be seen as a fiber just as much as types can (?)
A /\ B /\ C ===> D

|- goal(p)  as a marker. Always provable goal(p) == True
 
unify(pf1, pf2)

Rather than an equalizer, a pullback. t1 as a substituion mapping anyway?
G1 -> G2
G3 -> G2
Two telescope mappings that go to common context G2. Can I build a pullback?




In [ ]:
from dataclasses import dataclass

@dataclass
class Telescope():
    """ x | A(x), y | B(x,y), ... """
    vs : list[smt.ExprRef]
    preds : list[smt.BoolRef]
    # preds : list[kd.Proof] ? |- ForAll([x], P1), ForAll([x,y], P1 => P2), ...  
    # No but the context should be proof recievining, not proof producing.

    def expr(self, P):
        print(self.vs, self.preds, P)
        return kd.QForAll(self.vs, smt.And(self.preds), P)
    


x,y,z = smt.Reals("x y z")
Telescope([x,y], [x > 0, y > x])

@dataclass
class TeleSubst():
    dom : Telescope
    cod : Telescope
    subst: dict[smt.ExprRef, smt.ExprRef]
    # pf : list[smt.Proof] # same length as dom.preds

    def check(self,by=[]):
        pfs = []
        pfs.extend(by)
        for P in self.dom.preds:
            pfs.append(kd.prove(self.cod.expr(smt.substitute(P, *self.subst.items())), by=pfs))
        return pfs
    def __call__(self, P):
        # P is in context dom
        return smt.substitute(P, *self.subst.items())
    def __matmul__(self, other):
        # substutition forms a category
        assert self.dom == other.cod
        TeleSubst(other.dom, self.cod, {smt.substitute(...)})

# allows subtype weakening.
# if it only allowed syntactic substitutions, then we don't really need the proofs

ctx1 = Telescope([x,z], [x >= 0, z >= x])
ctx2 = Telescope([y], [y >= -1])
TeleSubst(ctx1, ctx2, {x: y + 1, z : y + 10}).check()


# is there a unification procedure for making TeleSubst?
# G1 |- t1 =? t2 -| G2
# result is G1 -> G2 substitution? It's assymmetric?
# doesn't seem like the right shape.



[y] [y >= -1] y + 1 >= 0
[y] [y >= -1] y + 10 >= y + 1


[|- ForAll(y, Implies(And(y >= -1), y + 1 >= 0)),
 |- ForAll(y, Implies(And(y >= -1), y + 10 >= y + 1))]

In [ ]:
from kdrag.all import *

def subst(pf : kd.Proof, vs : list[smt.ExprRef], subst : list[smt.ExprRef]) -> kd.Proof:
    vs1, ab = kd.kernel.herb(smt.ForAll(vs, smt.substitute_vars(pf.thm.body(), *reversed(subst))))
    a = kd.kernel.instan([smt.substitute(t, *zip(vs, vs1)) for t in subst], pf)
    return kd.kernel.modus(ab, a)
x,y,z = smt.Reals("x y z")
p = kd.prove(smt.ForAll([x,z], smt.And(z == z, x == x)))
subst(p, [y, z], [y + 1, z])

def weaken(pf, x):


def unify(pf1, pf2) -> Optional[kd.Proof]:
    # unify the two proofs forall x, p1(x) and forall y, p2(y)
    # return a new proof
    vs1, p1 = kd.utils.open_binder(pf1.thm)
    vs2, p2 = kd.utils.open_binder(pf2.thm)
    vs = vs1 + vs2
    subst = kd.utils.unify(vs1+vs2,p1,p2)
    if subst is not None:
        vs = set(vs) - set(subst.keys()) # maybe keep 'em ordered
        return subst_tac(pf1, vs, )

def build_telescope(pf : kd.Proof, vs, hyps, conc):
    # find a substitution based on unifying conc


        

def chain(pf1, pf2):
    # decompose into rules.
    # unify
    # rebuild






|- ForAll([y, z], And(z == z, y + 1 == y + 1))

In [3]:
import kdrag.reflect as reflect
from kdrag.all import *
x = smt.Bool("x")
@reflect.reflect
def fact(x : int) -> int:
    if x <= 0:
        return 1
    else:
        return x*fact(x-1)
fact.defn

|- ForAll(x, fact(x) == If(0 >= x, 1, x*fact(x - 1)))

# verilog extract

Silviu makes a good point.
If you're going to be comparing test case outputs,
No need to 



In [ ]:


x,y,z = smt.Bools("x y z")
a,b,c = smt.BitVecs("a b c", 8)
print(to_verilog("test", [x,y], {z : x & y}))
print(to_verilog("test", [a,b], {z : a & b + 7 != b}))
#(a & b != b).decl().name()

def test():
    with open("/tmp/test.v", "w") as f:
        f.write(to_verilog("test", [x,y], {z : x & y}))
    


// Generated by knuckledragger
module test (
   input x,
   input y,
   output z
);
assign z = (x && y);
endmodule

// Generated by knuckledragger
module test (
   input [7:0] a,
   input [7:0] b,
   output z
);
assign z = ((a & (b + 8'd7)) != b);
endmodule



To say that a verilog file has a property is an axiom schema?

In [ ]:
def sby_axiom(modfile, name, ins, outs):
    


# diff




In [ ]:
from kdrag.all import *

def diff(e, x):
    if e.eq(x):
        return 1
    elif smt.is_value(e):
        return 0
    elif smt.is_add(e):
        return smt.Sum([diff(c,x) for c in e.children()])
    elif e.decl() == real.sin:
        return real.cos(e) * diff(e, x)
    elif e.decl() == real.cos:
        return -real.sin(e) * diff(e, x)
    elif e.decl() == real.exp:
        return real.exp(e) * diff(e, x)
    
    


# knuckelproblems


In [6]:


from kdrag.all import *
# Knuckledragger support algebraic datatypes and induction
Nat = kd.Inductive("MyNat")
Zero = Nat.declare("Zero")
Succ = Nat.declare("Succ", ("pred", Nat))
Nat = Nat.create()
# We can define an addition function by cases
n,m = smt.Consts("n m", Nat)
add = smt.Function("add", Nat, Nat, Nat)
add = kd.define("add", [n,m], 
    kd.cond(
        (n.is_Zero, m),
        (n.is_Succ, Nat.Succ(add(n.pred, m)))
))

"""
add = kd.define("add", [n,m], 
    n.match_(
        (Nat.Zero, m),
        (Nat.Succ(n), Nat.Succ(add(n, m)))
))
add.defn
"""
add.defn


|- ForAll([n, m],
       add(n, m) ==
       If(is(Zero, n),
          m,
          If(is(Succ, n),
             Succ(add(pred(n), m)),
             unreachable!6)))

In [9]:
import kdrag.reflect as reflect

@reflect.reflect
def fact(x : int) -> int:
    if x <= 0:
        return 1
    else:
        return x*fact(x-1)
fact.defn

|- ForAll(x, fact(x) == If(0 >= x, 1, x*fact(x - 1)))

# vectors

Bitvector operations

V4 style of 


In [ ]:
BV32 = smt.BitVecSort(32)




# Type resgistry

Some 

add_assoc
add_comm
mul_assoc
mul_comm


COuld reaplce sortdispatch using T.add, T.mul and just have ExprRef.__add__ = lambda x,y: x.sort().add(x,y)



BoolSort
RealSort
StringSort
etc


In [82]:
from kdrag.all import *
T = smt.DeclareSort("T")
T1 = smt.DeclareSort("T")
id(T1) == id(T)
T1.get_id() == T.get_id()

True

In [5]:
from kdrag.all import *
smt.sort_registry
import kdrag.theories.nat as nat
smt.sort_registry
import kdrag.theories.bitvec as bv
smt.sort_registry

{2147483659: Int,
 2147483648: Bool,
 2147483658: Real,
 2147483815: NatI,
 2147483814: Nat,
 2147483704: BitVec(1),
 2147483711: BitVec(8),
 2147483898: Seq(BitVec(1)),
 2147483841: BitVecN}

In [ ]:
# hash cons the types so that we can tag useful stuff on them
types = {}

def DeclareSort(name):
    if name not in types:
        types[name] = type(name, (object,), {})
    return types[name]


# Schematic Vars
Explicitly pulling schematic variables out, may let me refer to them in subproofs (How?)

   G |- A
------------- weaken
     G, v fresh |- A
Well that's annoying.

app(forall x, A, t)




forall x, A  

Maybe I should consider this as A Proof extension rather than proof replacement, likke refinement types


```
Gam |- t   x not in t
-----------
Gam, x |- t 

```


If I merged forall intro and removal


This is just |- forall x, P(x) but with some extra junk.

capture avoiding

https://ncatlab.org/nlab/show/substitution

https://en.wikipedia.org/wiki/Admissible_rule
https://cstheory.stackexchange.com/questions/54600/admissible-rules-in-dependent-type-theory

Gam |- A then sigma Gam |- sigma A


The point was that

prove(t, )

def cprove(vs, p, by=)


a curried form of backward proof tracking.
modus for leaves
comp for moves.
but yeah, split
A /\ B
(A => B => A /\ B)
(A => A \/ B)
(B => A \/ B)

A => B => C

but then aplpying move in context is annoying. Hmm.
And I don't give a shit about anything except quantifier moves so it's wasteful effort.

Maybe the callback method is easier. But it'll have so many expensive calls


def intros():
   




In [ ]:
def modus(ab, a):
    assert smt.is_implies(ab) and ab.arg(0).eq(a)
    assert isinstance(ab, Proof) and isinstance(a, Proof)
    return Proof(ab.arg(1), reasons=[ab, a])

def impl_refl(a):
    return Proof(smt.Implies(a,a))

def comp(ab, bc): # cut sort of
    assert smt.is_implies(ab) and smt.is_implies(bc) and ab.arg(1).eq(bc.arg(0))
    assert isinstance(ab, Proof) and isinstance(bc, Proof)
    return Proof(smt.Implies(ab.arg(0), bc.arg(1)), reasons=[ab, bc])


class Lemma():
    self.backward_proof = impl_refl(goal)
    self.topgoal = goal
    self.curgoal = goal
    

def qed():
    assert self.backwardproof.thm.eq(self.topgoal)
    return self.backwardproof
def fixes():
    herb(self.topgoal)
    self.backwardproof = comp(  , self.backwardproof)(modus(ab,a)))
def prove():
    p = self.prove(curgoal)
    self.backwardproof = modus(self.backwardproof, self.proof)
def intros():
    self.curgoal = self.curgoal.arg(0)
    # goal is unchanged?
    # (a -> b) -> topgoal. uhhh. G -> 
    # 
    # /\ (forall vs, hyps => conc) -> topgoal
    # (hyp1 => conc1) => (hyp2 => conc2) => hyp3 => topgoal
    # goal1 => goal2 => topgoal



Goal
  def to_expr
   

                                                


In [ ]:
def deinstan(pf):
    # get substution if last thing was an instan?
    # Super weird idea.

def cprove(
    vs : list[smt.ExprRef],
    thm: smt.BoolRef,
    by = lambda *args: [] #Callable[smt.ExprRefglist[kd.Proof],
    **kwargs
) -> kd.kernel.Proof:
    """Contextual proof. Open up variable binders and give by lemmas access to them in callback"""
    goal = smt.ForAll(vs, thm)
    vs1, pf = kd.kernel.herb(goal)
    subproof = kd.prove(pf.thm.arg(0), by(*vs1))
    return kd.kernel.modus(subproof, pf)



def indprove(v, thm):
    """Inductive proof. Open up variable binders and give by lemmas access to them in callback"""
    goal = smt.ForAll(v, thm)
    vs1, pf = kd.kernel.herb(goal)
    subproof = kd.prove(pf.thm.arg(0), by(*vs1))
    return kd.kernel.modus(subproof, pf)

def indprove(v : smt.ExprRef, thm, by=None, using=None):
    l = kd.Lemma(smt.ForAll(v, thm))
    v1 = l.fix()
    l.induct(v1, using=using)
    l.auto(by=by)
    return l.qed()

In [ ]:
def subst(vs, p : kd.Proof, subst):
    assert isinstance(p, kd.Proof)
    vs1, body = kd.utils.open_binder_unhygienic(p.fm)
    assert kd.utils.free_in(vs - vs1, body)

# vs

# ok so we can achieve of substutiton by combining herb and instan.
def subst(vs, p : kd.Proof, subst):
    goal = smt.ForAll(vs, smt.substitute(p.fm, *subst.items()))
    l = kd.Lemma(goal)
    vs = l.fixes()
    l.qed(by=p(subst[v] for v in vs))
    #vs, pf = kd.kernel.herb(p)



In [ ]:
class Proof():
    ctx : dict[smt.SortRef, int] # How many de bruijn indices are allowed?
    fm : smt.BoolRef
    

In [ ]:
class Proof():
    vs : list[smt.ExprRef] # ctx
    fm : smt.BoolRef | smt.QuantifierRef
    reasons : list[object]
    def __call__(self, *args, vs=[]):
        smt.substitute(self.fm, *zip(self.vs, args))

def free(vs, fm):
    #fm2 = smt.substitute(fm, *[(v, smt.FreshConst(v.sort())) for v in vs])
    return smt.Lambda(vs, fm).body().eq(fm)
    #return fm2.eq(fm)

def lift(p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    return Proof(
        vs = [],
        fm = p.fm,
        reasons = [p]
    )
def lift_forall(p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    vs, fm = kd.utils.open_binder_unhiegenic(p.fm)
    return Proof(
        vs = vs,
        fm = fm,
        reasons = [p]
    )
def lift_free(ctx, p : kd.Proof) -> Proof:
    assert isinstance(p, kd.Proof)
    assert all(v in ctx for v in p.vs)
    assert free(ctx, p.fm)
    return Proof(
        vs = ctx,
        fm = p.fm,
        reasons = [p]
    )

"""
  Gam |- forall x, P
---------------
 Gam, x |- P

 This is reversed intro rule. Feels odd.
"""
def open(p : Proof) -> Proof:
    vs, fm = kd.utils.open_binder_unhiegenic(p.fm)
    Proof(p.vs + vs, fm, [p])

def intro(n, p : Proof):
    return Proof(p.vs[n:], smt.ForAll(p.vs[:n], p.fm), [p])

def extend(vs, p : Proof) -> Proof:
    s = smt.Solver()
    s.add(smt.ForAll(p.vs, p.fm))
    s.add(smt.Not(smt.ForAll(p.vs + vs, p.fm)))
    res = s.check()
    if res == smt.unsat:
        return Proof(
            vs = p.vs + vs,
            fm = p.fm,
            reasons = [p]
        )
    else:
        raise Exception("Unknown result from solver")


def subst(p : Proof, ctx : list[smt.ExprRef], subst : dict[smt.ExprRef, smt.ExprRef]):
    assert isinstance(p, Proof)
    assert all(v in p.vs for v in  subst.keys())
    assert free(ctx - p.ctx, p.fm)
    #fm = smt.substitute(p.fm, *[(v, smt.FreshConst(v.sort())) for v in ctx])
    fm = smt.substitute(p.fm, *subst.items())
    # I need to check that ctx2 does not appear in fm. Otherwise I could |- even(zero) ---> zero |- even(zero)
    return Proof(
        vs = ctx,
        fm = fm,
        reasons = [p]
    )


"""
G |- p1    G |- p2 ... 
-------
    G |- p

"""

def prove(ctx, expr, by=None):
    allctx = set(ctx)
    for p in by:
        assert isinstance(p, Proof)
        assert p.ctx == ctx
    s.add(smt.Not(smt.ForAll([ctx], smt.Implies(smt.And(hyp.fm for hyp in by), expr))))
    s.check()




Ok so yes, any forall infecting the lemmas is a problem.
And some things get foralls because its hard an annoying 

qed should 


In [6]:
from kdrag.all import *
import kdrag.theories.real as real
x,y = smt.Reals("x y")
cos = real.cos
sin = real.sin
#os_diff = kd.prove(smt.ForAll([x,y], cos(x - y) == cos(x)*cos(y) + sin(x)*sin(y)), [real.cos_add, real.cos_neg, real.sin_neg])

_l = kd.Lemma(smt.ForAll([x,y], cos(x - y) == cos(x)*cos(y) + sin(x)*sin(y)))
_x, _y = _l.fixes()
_l.symm()
_l.eq(cos(_x + (-_y)))
#_l.rw(kd.prove(cos(_x - _y) == cos(_x + (-_y))))
#_l.rw(kd.prove(smt.ForAll([x,y], x - y == (x + (-y))))(_x, _y))
_l.rw(real.cos_add(_x, -_y))
#_l.auto(by=[real.cos_add, real.sin_add])
_l.auto(by=[real.cos_neg(_y), real.sin_neg(_y)])
_l.lemmas
_l.qed()
cos_diff = _l.lemmas


# Homomorphism theorems


In [ ]:
from kdrag.all import *
import kdrag.property as prop
import kdrag.theories.algebra.group as group

A = smt.DeclareSort('A')
def Group(A):
    mul = smt.Function('mul', A, A, A)
    inv = smt.Function('inv', A, A)
    id_ = smt.Const('id', A)
    x,y,z = smt.Consts('x y z', A)
    kd.notation.mul.register(A, mul)
    kd.notation.invert.register(A, inv)
    assoc = kd.axiom(smt.ForAll([x,y,z], x * (y * z) == (x * y) * z))
    id_left = kd.axiom(smt.ForAll([x], id_ * x == x))
    inv_left = kd.axiom(smt.ForAll([x], inv(x) * x == id_))
    return id_, assoc, id_left, inv_left


B = smt.DeclareSort('B')
group.Group.register(A)
idA, assocA, id_leftA, inv_leftA = Group(A)
idB, assocB, id_leftB, inv_leftB = Group(B)

h = smt.Function('h', A, B)
def homo(id_, h):
    A = h.domain(0)
    B = h.range()
    x,y,z = smt.Consts("x y z", A)
    mul = smt.ForAll([x,y], h(x * y) == h(x) * h(y))
    id_ = h(idA) == idB
    inv = smt.ForAll([x], h(~x) == ~h(x))
    return mul, id_, inv
homo(h)





ValueError: ('Not registered in typeclass', (A,))

Universe of groups.

I was saying construct it as permuations  Int -> Int

Nontrivial
But we could also driectly axiomatize

* is composition
id is  x -> x
homomorphisms need to... be careful here. Only push homo through on particular Group?
SetGrp 





In [ ]:

Grp = smt.DeclareSort("Grp")
SetGrp = set_.SetSort(Grp)
inv = smt.Function("inv", Grp, Grp)
# inv = smt.Function("inv", GrpSet, GrpSet)
mul = smt.Function("mul", Grp, Grp) # We just leave mul un

Closed = smt.QForAll([x,y, S], S[x], S[y], S[x * y], S[inv(x)])

smt.QForAll([S], Closed[S], S[x], S[y], S[z],  x*(y*z) == (x * y) * z) # Is there a model where mul can be associative and always defined? 
smt.QForAll([S], Closed[S], S[x], inv(x) * x == x)  # maybe unconditionally actually
smt.QForAll([S], id_ * x == x) # also unconditiojnality